# Imports

In [2]:
# import matplotlib.pyplot as plt
# import seaborn as sns

In [4]:
import nbconvert
from IPython.display import HTML

In [5]:
# 34234
import pandas as pd
from sqlalchemy import create_engine
import psycopg2 as pg
from datetime import datetime
import json
import numpy as np
from plotly.subplots import make_subplots
import plotly.express as px
import plotly.graph_objects as go
import plotly.io as pio
import datetime
pio.renderers.default = 'notebook'
import warnings
import numpy_financial as npf

warnings.filterwarnings("ignore")
from datetime import timedelta
import numpy_financial as npf

ModuleNotFoundError: No module named 'sqlalchemy'

In [4]:
from sqlalchemy import create_engine
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pickle
import json
from tqdm import tqdm
import numpy as np
import warnings
from datetime import datetime, timedelta
import ast

import snowflake.connector
from snowflake.sqlalchemy import URL
from sqlalchemy import create_engine

In [5]:
from tqdm import tqdm
from sklearn.decomposition import PCA
from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler
from hampel import hampel
from queries import *

In [6]:
pd.set_option('display.max_columns', 500)
pd.set_option('display.max_rows', 500)

In [7]:
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

def plot_hists(df, columns_list):
    # Define the columns to plot
    columns_to_plot = columns_list
    
    # Determine the height of each subplot
    subplot_height = 4  # Height of each subplot in inches
    total_height = subplot_height * len(columns_to_plot)

    # Create a figure with subplots for each column
    fig, axes = plt.subplots(nrows=len(columns_to_plot), figsize=(7, total_height))

    # Plot each column as a histogram on its corresponding subplot
    for i, column in enumerate(columns_to_plot):
        # Calculate the 95th percentile for the column
        percentile_95 = np.percentile(df[column].dropna(), 95)
        
        # Clip the values at the 95th percentile
        clipped_data = df[column].clip(upper=percentile_95)
        
        # Plot the histogram
        sns.histplot(x=clipped_data, ax=axes[i], kde=True)
        axes[i].set_title(f'{column} (clipped at 95th percentile)')

    # Adjust the layout and spacing of the subplots
    fig.tight_layout()

    # Display the chart
    plt.show()


In [8]:
def query_for_data(query):
    engine = create_engine(URL(user=my_user,password=my_pass,account=my_account,warehouse=my_warehouse,database=my_db))
    connection = engine.connect()
    df = pd.read_sql(query, connection)
    return df

In [9]:
sns.set_theme()

# Load Data

In [11]:
my_user="AMIT_S"
my_pass="Amit2525!!"
my_account="xukvgcv-xm56910"
my_warehouse='ANALYTICS_WH'
my_db='ghana_prod'


In [14]:
query_recent ='''
select ml.ln,xc.*,xc.start_time::date as call_date,
from public.xcally_call_records xc
left join ml.loan_info_tbl ml on ml.loan_id=xc.loan_id
where start_time::date between '2024-09-28' and '2024-11-30'
'''
xc_data = query_for_data(query_recent)

In [15]:
query2 ='''
SELECT ml.client_id,lb.*, ml.LOAN_ID  FROM RISK_REPORTS.LOANBOOK_HISTORY lb
LEFT JOIN ml.LOAN_INFO_TBL ml ON ml.LOAN_KEY = lb.LOAN_KEY 
where lb.period::date  between '2024-09-28' and '2024-11-30'
'''
lb_hist = query_for_data(query2)

In [16]:
query3 ='''
SELECT DISTINCT *,enterat::date as call_date  FROM ghana_prod.DATA.XCALLY_REPORT_MEMBER
WHERE data1 IN ('inbound','outbound')
and enterat::date  between '2024-09-28' and '2024-11-30'
'''
extra_calls = query_for_data(query3)

In [25]:
query4 ='''
SELECT DISTINCT c.ID AS client_id ,sms.SENT_TO ,sms.SENT_AT,sms.SENT_AT::date as call_date,
sms.STATUS_GROUPNAME,sms.messageid  FROM ghana_prod.DATA.INFOBIP_SMS sms
LEFT JOIN ghana_prod.MAMBU. CLIENT c ON (c.MOBILEPHONE1 = sms.SENT_TO OR c.MOBILEPHONE2 = sms.SENT_TO)
WHERE SENT_AT::date  between '2024-09-28' and '2024-11-30'
AND client_id IS NOT NULL 

'''
sms_data = query_for_data(query4)

# Prep Data 

In [27]:
date_range = pd.date_range(start='2024-09-28', end='2024-11-30')
# date_range = pd.date_range(start='2024-7-01', end=datetime.today().strftime('%Y-%m-%d'))

### xcally

In [29]:
xc_orig=xc_data

In [30]:
# xc_data=xc_orig

In [31]:
xc_data.shape

(5974114, 25)

In [32]:
xc_data['call_date']=pd.to_datetime(xc_data['call_date'])

In [33]:
xc_data['call_week'] = xc_data['call_date'].dt.to_period('W').apply(lambda r: r.start_time.strftime('%Y-%m-%d'))
xc_data['call_month'] = xc_data['call_date'].dt.to_period('M').apply(lambda r: r.start_time.strftime('%Y-%m-%d'))
xc_data['call_weekday'] = xc_data['start_time'].apply(lambda x: (x - pd.Timedelta(days=x.weekday())).strftime('%Y-%m-%d') if x.weekday() < 5 else None)


In [34]:
xc_data = xc_data.sort_values(by=['loan_id', 'start_time'])

In [35]:
xc_data['answered_ivr']=0
xc_data['answered_queue']=0
xc_data.loc[(xc_data.state_desc=='Answer') & (xc_data.call_type=='ivr'),'answered_ivr']=1
xc_data.loc[(xc_data.state_desc=='Answer') & (xc_data.call_type=='queue'),'answered_queue']=1

In [36]:
xc_data.call_date=pd.to_datetime(xc_data.call_date)

In [37]:
ans_df=pd.DataFrame(xc_data.groupby(['loan_id','call_date']).answered_queue.sum()).reset_index()
ans_df['start_time_plus1']=ans_df.call_date + pd.Timedelta(days=1)
ans_df['start_time_plus2']=ans_df.call_date + pd.Timedelta(days=2)
ans_df.head(2)

,loan_id,call_date,answered_queue,start_time_plus1,start_time_plus2
0,054218766,2024-11-02,0,2024-11-03,2024-11-04
1,079119394,2024-11-02,0,2024-11-03,2024-11-04


In [38]:
# # add all dates for each loan id
loan_ids = xc_data['loan_id'].unique()
all_combinations = pd.MultiIndex.from_product([loan_ids, date_range], names=['loan_id', 'call_date']).to_frame(index=False)
df_complete = pd.merge(all_combinations, xc_data, on=['loan_id', 'call_date'], how='left')


In [39]:
xc_data=df_complete

In [40]:
# # add answered yesterday
xc_calls_ext=pd.merge(xc_data,ans_df[['loan_id','call_date','start_time_plus1','answered_queue']],how='left',left_on=['loan_id','call_date'],right_on=['loan_id','start_time_plus1']).rename(columns={'answered_queue':'answered_yesterday'})
xc_calls_ext=xc_calls_ext.rename(columns={'answered_queue_y':'answered_yesterday','answered_queue_x':'answered_queue','call_date_x':'call_date'}).drop(columns=['call_date_y','start_time_plus1'])
xc_calls_ext.answered_yesterday=xc_calls_ext.answered_yesterday.fillna(0)

# add answered two days ago
xc_calls_ext=pd.merge(xc_calls_ext,ans_df[['loan_id','call_date','start_time_plus2','answered_queue']],how='left',left_on=['loan_id','call_date'],right_on=['loan_id','start_time_plus2']).rename(columns={'answered_queue_y':'answered_last_2_days'})
xc_calls_ext=xc_calls_ext.rename(columns={'answered_queue_x':'answered_queue','call_date_x':'call_date'}).drop(columns=['call_date_y','start_time_plus2'])
xc_calls_ext.answered_last_2_days=xc_calls_ext.answered_last_2_days.fillna(0)

# last fix
xc_calls_ext.loc[xc_calls_ext['answered_yesterday']==1,'answered_last_2_days']=1
xc_calls_ext.shape

(21611931, 32)

In [41]:
# xc_calls_ext[xc_calls_ext.loan_id=='HOZV707']

In [42]:
xc_data=xc_calls_ext

In [43]:
xc_data.campaign_name.value_counts()#.head(20)

campaign_name
GH_IVR_AUTOMATED_COMMUNITY_AND_RECOVERY_CAMPAIGN_UNANSWERED    996344
GH_Recovery_Unanswered_IVR_Campaign                            880021
GH_IVR_AUTOMATED_COMMUNITY_AND_RECOVERY_CAMPAIGN               755453
GH_Recovery_Special_IVR_Campaign                               665552
GH_Legal_Calls                                                 304919
GH_IVR_WAIVER_30_PERCENT_CAMPAIGN                              251213
GH_IVR_WAIVER_40_PERCENT_CAMPAIGN                              245380
GH_Mixed_Priority_Campaign_Mid                                 223167
GH_Mixed_Priority_Campaign                                     173075
backup_ghana_GH_manual_call_New_Model_B_dpd_0_30               158076
GH_Legal_Discount_Campaign                                     150409
GH_WRITTEN_OFF_IVR                                             125041
GH_Pay_Early_IVR_Campaign                                      124958
GH_Mixed_Priority_Campaign_Low                                  98660
backup

In [44]:
xc_data['campaign_short'] = np.select(
    [
        xc_data['campaign_name'].str.contains('GH_Initial_Recovery_Team', case=False, na=False),
        xc_data['campaign_name'].str.contains('mid', case=False, na=False),
        xc_data['campaign_name'].str.contains('low', case=False, na=False),
        xc_data['campaign_name'].str.contains('legal', case=False, na=False),
        xc_data['campaign_name'].str.contains('high', case=False, na=False),
        xc_data['campaign_name']=='GH_Mixed_Priority_Campaign'
    ],
    ['initial', 'mid_priority', 'low_priority', 'legal','high_priority','mixed_priority'],
    default='other'
)

In [45]:
xc_data['campaign_short'].value_counts()

campaign_short
other             20607691
legal               455328
mid_priority        223167
mixed_priority      173075
low_priority         98660
initial              54010
Name: count, dtype: int64

In [61]:
xc_data.start_time.isna().mean()

0.7235733354876989

In [75]:
xc_data.call_date=pd.to_datetime(xc_data.call_date)
xc_data['call_week'] = xc_data['start_time'].dt.to_period('W').apply(
    lambda r: r.start_time.strftime('%Y-%m-%d') if not pd.isnull(r) else None
)
xc_data['call_month'] = xc_data['start_time'].apply(
    lambda r: r.to_period('M').start_time.strftime('%Y-%m-%d') if not pd.isnull(r) else None
)
xc_data['call_hour'] = xc_data['start_time'].apply(
    lambda r: r.hour if not pd.isnull(r) else None
)

In [76]:
xc_data.head(2)

,loan_id,call_date,ln,id,member_name,caller_id_num,caller_id_name,scheduled_at,start_time,response_time,answer_time,state_desc,campaign_name,call_type,disposition,disposition_2,disposition_3,date,created_at,installment_identifier,phone,team,contact_id,data_origin,end_time,call_week,call_month,call_weekday,answered_ivr,answered_queue,answered_yesterday,answered_last_2_days,campaign_short,call_hour
0,054218766,2024-09-28,NaN,NaN,NaN,NaN,NaN,NaT,NaT,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaT,None,None,NaN,NaN,NaN,0.0,0.0,other,NaN
1,054218766,2024-09-29,NaN,NaN,NaN,NaN,NaN,NaT,NaT,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaT,None,None,NaN,NaN,NaN,0.0,0.0,other,NaN


In [77]:
lb_hist.period=pd.to_datetime(lb_hist.period)

In [78]:
xc_data=pd.merge(xc_data,lb_hist[['loan_id','period','days_late']],how='left',left_on=['loan_id','call_date'],right_on=['loan_id','period']).drop_duplicates()
xc_data.head(2)

,loan_id,call_date,ln,id,member_name,caller_id_num,caller_id_name,scheduled_at,start_time,response_time,answer_time,state_desc,campaign_name,call_type,disposition,disposition_2,disposition_3,date,created_at,installment_identifier,phone,team,contact_id,data_origin,end_time,call_week,call_month,call_weekday,answered_ivr,answered_queue,answered_yesterday,answered_last_2_days,campaign_short,call_hour,period,days_late
0,054218766,2024-09-28,NaN,NaN,NaN,NaN,NaN,NaT,NaT,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaT,None,None,NaN,NaN,NaN,0.0,0.0,other,NaN,NaT,NaN
1,054218766,2024-09-29,NaN,NaN,NaN,NaN,NaN,NaT,NaT,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaT,None,None,NaN,NaN,NaN,0.0,0.0,other,NaN,NaT,NaN


In [79]:
xc_data.answered_yesterday=np.clip(xc_data.answered_yesterday,0,1)
xc_data.answered_last_2_days=np.clip(xc_data.answered_last_2_days,0,1)

In [80]:
xc_data.answered_yesterday.value_counts()

answered_yesterday
0.0    21344223
1.0      267708
Name: count, dtype: int64

In [81]:
xc_data.answered_last_2_days.value_counts()

answered_last_2_days
0.0    21076275
1.0      535656
Name: count, dtype: int64

In [ ]:
#case when (days_late between 3 and 15 AND answered_yesterday=1) then 1 else 0 end as IS_EXCLUSION_DPD_3_15,
# case when (days_late between 16 and 30 AND answered_in_past_two_days=1) then 1 else 0 end as IS_EXCLUSION_DPD_16_30,

In [82]:
xc_data=xc_data.rename(columns={'answered_last_2_days':'answered_two_days'})

In [83]:
# xc_data['is_exclusion_DPD_0_M1_M3_M5'] = np.where(
#     (xc_data['call_date'] < pd.Timestamp('2024-05-28')) & 
#     (xc_data['days_late'].isin([-1, -3, -5, 0])) & 
#     (xc_data['answered_two_days'] == 1),
#     1,
#     0
# )

xc_data['is_exclusion_DPD_16_30'] = np.where(
    (xc_data['days_late'].between(16, 30)) & 
    (xc_data['answered_two_days'] == 1),
    1,
    0
)

xc_data['is_exclusion_DPD_3_15'] = np.where(
    (xc_data['days_late'].between(3, 15)) & 
    (xc_data['answered_yesterday'] == 1),
    1,
    0
)

In [84]:
xc_data.head(2)

,loan_id,call_date,ln,id,member_name,caller_id_num,caller_id_name,scheduled_at,start_time,response_time,answer_time,state_desc,campaign_name,call_type,disposition,disposition_2,disposition_3,date,created_at,installment_identifier,phone,team,contact_id,data_origin,end_time,call_week,call_month,call_weekday,answered_ivr,answered_queue,answered_yesterday,answered_two_days,campaign_short,call_hour,period,days_late,is_exclusion_DPD_16_30,is_exclusion_DPD_3_15
0,054218766,2024-09-28,NaN,NaN,NaN,NaN,NaN,NaT,NaT,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaT,None,None,NaN,NaN,NaN,0.0,0.0,other,NaN,NaT,NaN,0,0
1,054218766,2024-09-29,NaN,NaN,NaN,NaN,NaN,NaT,NaT,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaT,None,None,NaN,NaN,NaN,0.0,0.0,other,NaN,NaT,NaN,0,0


In [85]:
xc_data.call_date.max()

Timestamp('2024-11-30 00:00:00')

### extra calls

In [86]:
extra_calls.call_date=pd.to_datetime(extra_calls.call_date)
extra_calls['call_week'] = extra_calls['call_date'].dt.to_period('W').apply(lambda r: r.start_time.strftime('%Y-%m-%d'))
extra_calls['call_month'] = extra_calls['call_date'].dt.to_period('M').apply(lambda r: r.start_time.strftime('%Y-%m-%d'))

In [87]:
extra_calls.head(2)

,id,channel,membername,interface,type,duration,enterat,exitat,data1,data2,data3,data4,data5,role,internal,uniqueid,createdat,updatedat,call_date,call_week,call_month
0,4417017,voice,IbrahimMusah,SIP/IbrahimMusah,TALKING,31,2024-11-25 09:42:22,2024-11-25 09:42:53,outbound,1023,413233244041358,None,None,agent,1023,1732527735.49329,2024-11-25 09:42:53,2024-11-25 09:42:53,2024-11-25,2024-11-25,2024-11-01
1,4417134,voice,IbrahimMusah,SIP/IbrahimMusah,TALKING,87,2024-11-25 09:48:05,2024-11-25 09:49:32,outbound,1023,413233244041358,None,None,agent,1023,1732528078.50752,2024-11-25 09:49:32,2024-11-25 09:49:32,2024-11-25,2024-11-25,2024-11-01


In [88]:
extra_calls.shape,extra_calls.id.nunique()

((2237, 21), 2237)

In [89]:
add2=pd.DataFrame(extra_calls[extra_calls.data1=='outbound'].groupby('call_date').id.nunique()).rename(columns={'id':'n_manual_outbound_calls'})
add2['n_inbound_calls']=extra_calls[extra_calls.data1=='inbound'].groupby('call_date').id.nunique()
add2=add2.fillna(0)
# add2

In [90]:
add2.index=add2.index.astype('str')

In [91]:
add22=pd.DataFrame(extra_calls[extra_calls.data1=='outbound'].groupby('call_week').id.nunique()).rename(columns={'id':'n_manual_outbound_calls'})
add22['n_inbound_calls']=extra_calls[extra_calls.data1=='inbound'].groupby('call_week').id.nunique()
add22=add22.fillna(0)
add22

,n_manual_outbound_calls,n_inbound_calls
call_week,,
2024-09-23,41,0.0
2024-09-30,233,0.0
2024-10-07,98,0.0
2024-10-14,33,0.0
2024-10-21,101,0.0
2024-10-28,735,0.0
2024-11-04,192,0.0
2024-11-11,175,0.0
2024-11-18,97,0.0


In [92]:
add22.index=add22.index.astype('str')

In [93]:
add222=pd.DataFrame(extra_calls[extra_calls.data1=='outbound'].groupby('call_month').id.nunique()).rename(columns={'id':'n_manual_outbound_calls'})
add222['n_inbound_calls']=extra_calls[extra_calls.data1=='inbound'].groupby('call_month').id.nunique()
add222=add222.fillna(0)
add222

,n_manual_outbound_calls,n_inbound_calls
call_month,,
2024-09-01,103,0.0
2024-10-01,1037,0.0
2024-11-01,1097,0.0


In [94]:
add222.index=add222.index.astype('str')

### lb hist

In [96]:
lb_hist.period=pd.to_datetime(lb_hist.period)
lb_hist['call_week'] = lb_hist['period'].dt.to_period('W').apply(lambda r: r.start_time.strftime('%Y-%m-%d'))
lb_hist['call_month'] = lb_hist['period'].dt.to_period('M').apply(lambda r: r.start_time.strftime('%Y-%m-%d'))

In [97]:
lb_hist.head(3)

,client_id,period,loan_key,days_late,loan_balance,loan_id,call_week,call_month
0,440882883,2024-11-28,8a1945ed91a0d8420191a1d08674592a,31,1952.11,BXNR582,2024-11-25,2024-11-01
1,440882883,2024-10-21,8a1945ed91a0d8420191a1d08674592a,-7,1586.49,BXNR582,2024-10-21,2024-10-01
2,440882883,2024-11-29,8a1945ed91a0d8420191a1d08674592a,32,1954.34,BXNR582,2024-11-25,2024-11-01


In [98]:
lb_hist.loc[lb_hist.days_late<0,'dpd_group']='dpd_less_than_0'
lb_hist.loc[lb_hist.days_late.between(0,2),'dpd_group']='dpd_0_2'
lb_hist.loc[lb_hist.days_late.between(3,30),'dpd_group']='dpd_3_30'
lb_hist.loc[lb_hist.days_late>30,'dpd_group']='dpd_31_plus'

In [105]:
add_lb=pd.DataFrame(lb_hist.groupby('period').dpd_group.value_counts()).reset_index()
add_lb=add_lb.rename(columns={'count':'n_loans'})
add_lb.head(2)

,period,dpd_group,n_loans
0,2024-09-28,dpd_less_than_0,51775
1,2024-09-28,dpd_3_30,12510


In [107]:
df_pivot = add_lb.pivot(index='period', columns='dpd_group', values='n_loans')

# Rename columns for clarity (optional)
df_pivot.columns = ['n_loans_' + col for col in df_pivot.columns]

# Reset index to make 'period' a column again
df_pivot.reset_index(inplace=True)

In [109]:
df_pivot=df_pivot.fillna(0)

In [113]:
add_lb=df_pivot
# add_lb

### sms

In [120]:
#sms dpd -3,-1,0,1,3,7,18,25

In [116]:
sms_data.shape

(6804735, 6)

In [118]:
sms_data.call_date=pd.to_datetime(sms_data.call_date)
sms_data['call_date']= sms_data['sent_at'].dt.to_period('D').apply(lambda r: r.start_time.strftime('%Y-%m-%d'))
sms_data['call_week'] = sms_data['sent_at'].dt.to_period('W').apply(lambda r: r.start_time.strftime('%Y-%m-%d'))
sms_data['call_month'] = sms_data['sent_at'].dt.to_period('M').apply(lambda r: r.start_time.strftime('%Y-%m-%d'))

In [119]:
sms_data.call_date=pd.to_datetime(sms_data.call_date)

In [120]:
sms_final=pd.merge(sms_data,lb_hist,how='left',left_on=['client_id','call_date'],right_on=['client_id','period'])
sms_final[sms_final.period.notna()].head(2)

,client_id,sent_to,sent_at,call_date,status_groupname,messageid,call_week_x,call_month_x,period,loan_key,days_late,loan_balance,loan_id,call_week_y,call_month_y,dpd_group
8,027960610,233545288443,2024-11-08 13:31:23,2024-11-08,None,140110813312336204,2024-11-04,2024-11-01,2024-11-08,8a192993926b19fd01926d182e001167,1.0,334.07,PRPO908,2024-11-04,2024-11-01,dpd_0_2
10,476550669,233245200663,2024-10-29 12:48:48,2024-10-29,None,136102912484801765,2024-10-28,2024-10-01,2024-10-29,8a19368c92484e9001924b7be4c81cc0,-3.0,2206.00,JQPH513,2024-10-28,2024-10-01,dpd_less_than_0


In [121]:
# sms_final=sms_final.rename(columns={'call_week_x':'call_week','call_month_x':'call_month'})

In [122]:
sms_final=sms_final.drop_duplicates()

In [123]:
sms_final.shape

(6811223, 16)

In [124]:
sms_final.head(2)

,client_id,sent_to,sent_at,call_date,status_groupname,messageid,call_week_x,call_month_x,period,loan_key,days_late,loan_balance,loan_id,call_week_y,call_month_y,dpd_group
0,767988965,233549436169,2024-11-05 18:30:20,2024-11-05,None,138110518302059061,2024-11-04,2024-11-01,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,355711492,233556609565,2024-10-28 18:30:09,2024-10-28,None,150102818300985013,2024-10-28,2024-10-01,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [125]:
add3=pd.DataFrame(sms_final.groupby('call_date').messageid.nunique()).rename(columns={'messageid':'n_sms_sent'})
add3['n_sms_sent_dpd_m3']=sms_final[sms_final.days_late==-3].groupby('call_date').messageid.nunique()
add3['n_sms_sent_dpd_m1']=sms_final[sms_final.days_late==-1].groupby('call_date').messageid.nunique()
add3['n_sms_sent_dpd_0']=sms_final[sms_final.days_late==0].groupby('call_date').messageid.nunique()
add3['n_sms_sent_dpd_1']=sms_final[sms_final.days_late==1].groupby('call_date').messageid.nunique()
add3['n_sms_sent_dpd_3']=sms_final[sms_final.days_late==3].groupby('call_date').messageid.nunique()
add3['n_sms_sent_dpd_7']=sms_final[sms_final.days_late==7].groupby('call_date').messageid.nunique()
add3['n_sms_sent_dpd_18']=sms_final[sms_final.days_late==18].groupby('call_date').messageid.nunique()
add3['n_sms_sent_dpd_25']=sms_final[sms_final.days_late==25].groupby('call_date').messageid.nunique()

add3.head(2)

,n_sms_sent,n_sms_sent_dpd_m3,n_sms_sent_dpd_m1,n_sms_sent_dpd_0,n_sms_sent_dpd_1,n_sms_sent_dpd_3,n_sms_sent_dpd_7,n_sms_sent_dpd_18,n_sms_sent_dpd_25
call_date,,,,,,,,,
2024-09-28,105794,1410.0,NaN,NaN,2194.0,691.0,NaN,38.0,232.0
2024-09-29,216036,10414.0,17116.0,NaN,NaN,1763.0,NaN,55.0,40.0


In [126]:
add3=add3.fillna(0)

In [127]:
add3.index=add3.index.astype('str')

In [128]:
sms_final=sms_final.rename(columns={'call_week_x':'call_week'})

In [129]:
add33=pd.DataFrame(sms_final.groupby('call_week').messageid.nunique()).rename(columns={'messageid':'n_sms_sent'})
add33['n_sms_sent_dpd_m3']=sms_final[sms_final.days_late==-3].groupby('call_week').messageid.nunique()
add33['n_sms_sent_dpd_m1']=sms_final[sms_final.days_late==-1].groupby('call_week').messageid.nunique()
add33['n_sms_sent_dpd_0']=sms_final[sms_final.days_late==0].groupby('call_week').messageid.nunique()
add33['n_sms_sent_dpd_1']=sms_final[sms_final.days_late==1].groupby('call_week').messageid.nunique()
add33['n_sms_sent_dpd_3']=sms_final[sms_final.days_late==3].groupby('call_week').messageid.nunique()
add33['n_sms_sent_dpd_7']=sms_final[sms_final.days_late==7].groupby('call_week').messageid.nunique()
add33['n_sms_sent_dpd_18']=sms_final[sms_final.days_late==18].groupby('call_week').messageid.nunique()
add33['n_sms_sent_dpd_25']=sms_final[sms_final.days_late==25].groupby('call_week').messageid.nunique()

add33.head(2)

,n_sms_sent,n_sms_sent_dpd_m3,n_sms_sent_dpd_m1,n_sms_sent_dpd_0,n_sms_sent_dpd_1,n_sms_sent_dpd_3,n_sms_sent_dpd_7,n_sms_sent_dpd_18,n_sms_sent_dpd_25
call_week,,,,,,,,,
2024-09-23,321830,11824,17116,NaN,2194,2454,NaN,93,272
2024-09-30,655369,7234,33003,3430.0,3759,6217,812.0,267,221


In [130]:
add33=add33.fillna(0)

In [131]:
add33.index=add33.index.astype('str')

In [132]:
sms_final=sms_final.drop(columns=['call_month_y']).rename(columns={'call_month_x':'call_month'})


In [133]:
add333=pd.DataFrame(sms_final.groupby('call_month').messageid.nunique()).rename(columns={'messageid':'n_sms_sent'})
add333['n_sms_sent_dpd_m3']=sms_final[sms_final.days_late==-3].groupby('call_month').messageid.nunique()
add333['n_sms_sent_dpd_m1']=sms_final[sms_final.days_late==-1].groupby('call_month').messageid.nunique()
add333['n_sms_sent_dpd_0']=sms_final[sms_final.days_late==0].groupby('call_month').messageid.nunique()
add333['n_sms_sent_dpd_1']=sms_final[sms_final.days_late==1].groupby('call_month').messageid.nunique()
add333['n_sms_sent_dpd_3']=sms_final[sms_final.days_late==3].groupby('call_month').messageid.nunique()
add333['n_sms_sent_dpd_7']=sms_final[sms_final.days_late==7].groupby('call_month').messageid.nunique()
add333['n_sms_sent_dpd_18']=sms_final[sms_final.days_late==18].groupby('call_month').messageid.nunique()
add333['n_sms_sent_dpd_25']=sms_final[sms_final.days_late==25].groupby('call_month').messageid.nunique()

add333.head(2)

,n_sms_sent,n_sms_sent_dpd_m3,n_sms_sent_dpd_m1,n_sms_sent_dpd_0,n_sms_sent_dpd_1,n_sms_sent_dpd_3,n_sms_sent_dpd_7,n_sms_sent_dpd_18,n_sms_sent_dpd_25
call_month,,,,,,,,,
2024-09-01,455998,14530,22108,1254,2194,3093,NaN,147,353
2024-10-01,2568250,36314,84026,9133,7987,16517,3830.0,1330,1140


In [134]:
add33=add33.fillna(0)

In [135]:
add33.index=add33.index.astype('str')

# Daily

In [154]:
df=xc_data

In [156]:
xc_data.head(3)

,loan_id,call_date,ln,id,member_name,caller_id_num,caller_id_name,scheduled_at,start_time,response_time,answer_time,state_desc,campaign_name,call_type,disposition,disposition_2,disposition_3,date,created_at,installment_identifier,phone,team,contact_id,data_origin,end_time,call_week,call_month,call_weekday,answered_ivr,answered_queue,answered_yesterday,answered_two_days,campaign_short,call_hour,period,days_late,is_exclusion_DPD_16_30,is_exclusion_DPD_3_15
0,054218766,2024-09-28,NaN,NaN,NaN,NaN,NaN,NaT,NaT,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaT,None,None,NaN,NaN,NaN,0.0,0.0,other,NaN,NaT,NaN,0,0
1,054218766,2024-09-29,NaN,NaN,NaN,NaN,NaN,NaT,NaT,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaT,None,None,NaN,NaN,NaN,0.0,0.0,other,NaN,NaT,NaN,0,0
2,054218766,2024-09-30,NaN,NaN,NaN,NaN,NaN,NaT,NaT,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaT,None,None,NaN,NaN,NaN,0.0,0.0,other,NaN,NaT,NaN,0,0


In [158]:
# # Calculate number of calls per team per day
calls_per_team = df.groupby(['call_date','call_type', 'campaign_short']).size().reset_index(name='n_calls_per_team').fillna(0)
calls_per_team = pd.pivot_table(calls_per_team, values='n_calls_per_team', index=['call_date', 'campaign_short'], columns=['call_type'], fill_value=0).reset_index()
calls_per_team['total'] = calls_per_team[['ivr', 'queue']].sum(axis=1)
calls_per_team.columns.name = None

calls_per_team.tail(5)

,call_date,campaign_short,ivr,queue,total
192,2024-11-28,other,103435.0,39176.0,142611.0
193,2024-11-29,legal,0.0,14343.0,14343.0
194,2024-11-29,other,78978.0,37830.0,116808.0
195,2024-11-30,legal,0.0,8214.0,8214.0
196,2024-11-30,other,59838.0,23480.0,83318.0


In [159]:
calls_per_hour_per_day = df.groupby(['call_date', 'call_hour']).size().reset_index(name='n_calls_per_hour')
ivr_calls_per_hour_per_day = df[df.call_type=='ivr'].groupby(['call_date', 'call_hour']).size().reset_index(name='n_ivr_calls_per_hour')
queue_calls_per_hour_per_day = df[df.call_type=='queue'].groupby(['call_date', 'call_hour']).size().reset_index(name='n_queue_calls_per_hour')

In [160]:
# Create Res
res=pd.DataFrame(df.groupby('call_date').id.nunique()).rename(columns={'id':'n_calls'})
res['n_queue_calls']=df[df.call_type=='queue'].groupby('call_date').id.nunique()
res['n_ivr_calls']=df[df.call_type=='ivr'].groupby('call_date').id.nunique()
res = res.reset_index()
res['day_of_week'] = pd.to_datetime(res['call_date']).dt.strftime('%a')
res=res.set_index('call_date')
res['avg_calls_per_hour']=calls_per_hour_per_day.groupby('call_date').n_calls_per_hour.mean()
res['avg_ivr_calls_per_hour']=ivr_calls_per_hour_per_day.groupby('call_date').n_ivr_calls_per_hour.mean()
res['avg_queue_calls_per_hour']=queue_calls_per_hour_per_day.groupby('call_date').n_queue_calls_per_hour.mean()

res['n_calls_ln0']=df[df.ln==0].groupby('call_date').id.nunique()
res['n_calls_ln0_queue']=df[(df.ln==0)&(df.call_type=='queue')].groupby('call_date').id.nunique()
res['n_calls_ln0_ivr']=df[(df.ln==0)&(df.call_type=='ivr')].groupby('call_date').id.nunique()
res.head(2)

,n_calls,n_queue_calls,n_ivr_calls,day_of_week,avg_calls_per_hour,avg_ivr_calls_per_hour,avg_queue_calls_per_hour,n_calls_ln0,n_calls_ln0_queue,n_calls_ln0_ivr
call_date,,,,,,,,,,
2024-09-28,72468,24163.0,48305,Sat,5176.285714,3450.357143,2013.583333,18848,4366.0,14482
2024-09-29,24317,NaN,24317,Sun,3473.857143,3473.857143,NaN,3228,NaN,3228


In [161]:
calls_per_team=calls_per_team.set_index('call_date')

In [162]:
def process_team_data(team_name, res_df, calls_df):
    team_data = calls_df[calls_df.campaign_short == team_name]
    team_data = team_data[['total', 'ivr', 'queue']]
    team_data = team_data.rename(columns={'total': f'n_calls_{team_name.lower().replace(" ", "_")}',
                                          'ivr': f'n_ivr_calls_{team_name.lower().replace(" ", "_")}',
                                          'queue': f'n_queue_calls_{team_name.lower().replace(" ", "_")}'})
    res_df = res_df.merge(team_data, right_index=True, left_index=True, how='left')
    return res_df

In [163]:
df.campaign_short.value_counts().index

Index(['other', 'legal', 'mid_priority', 'mixed_priority', 'low_priority',
       'initial'],
      dtype='object', name='campaign_short')

In [164]:
# Add Calls Per Team
teams = ['legal', 'mid_priority', 'initial', 'low_priority','high_priority','mixed_priority']
for team in teams:
    res = process_team_data(team, res, calls_per_team)

In [165]:
res.isna().mean()

n_calls                         0.000000
n_queue_calls                   0.250000
n_ivr_calls                     0.000000
day_of_week                     0.000000
avg_calls_per_hour              0.000000
avg_ivr_calls_per_hour          0.000000
avg_queue_calls_per_hour        0.250000
n_calls_ln0                     0.000000
n_calls_ln0_queue               0.250000
n_calls_ln0_ivr                 0.000000
n_calls_legal                   0.250000
n_ivr_calls_legal               0.250000
n_queue_calls_legal             0.250000
n_calls_mid_priority            0.656250
n_ivr_calls_mid_priority        0.656250
n_queue_calls_mid_priority      0.656250
n_calls_initial                 0.765625
n_ivr_calls_initial             0.765625
n_queue_calls_initial           0.765625
n_calls_low_priority            0.656250
n_ivr_calls_low_priority        0.656250
n_queue_calls_low_priority      0.656250
n_calls_high_priority           1.000000
n_ivr_calls_high_priority       1.000000
n_queue_calls_hi

In [166]:
# state desc
res['perc_calls_busy']=df[df.state_desc=='Busy'].groupby('call_date').id.nunique()/res.n_calls*100
res['perc_calls_no_answer']=df[df.state_desc=='NoAnswer'].groupby('call_date').id.nunique()/res.n_calls*100
res['perc_calls_congestion']=df[df.state_desc=='Congestion'].groupby('call_date').id.nunique()/res.n_calls*100
res['perc_calls_answer']=df[df.state_desc=='Answer'].groupby('call_date').id.nunique()/res.n_calls*100
res['perc_calls_abandoned']=df[df.state_desc=='Abandoned'].groupby('call_date').id.nunique()/res.n_calls*100
res['perc_calls_global']=df[df.state_desc=='Global'].groupby('call_date').id.nunique()/res.n_calls*100
res['perc_calls_no_such_number']=df[df.state_desc=='NoSuchNumber'].groupby('call_date').id.nunique()/res.n_calls*100
res['perc_calls_drop']=df[df.state_desc=='Drop'].groupby('call_date').id.nunique()/res.n_calls*100

#######
res['perc_calls_answer_queue']=df[(df.state_desc=='Answer') & (df.call_type=='queue')].groupby('call_date').id.nunique()/res.n_queue_calls*100
res['perc_calls_answer_ivr']=df[(df.state_desc=='Answer') & (df.call_type=='ivr')].groupby('call_date').id.nunique()/res.n_ivr_calls*100


In [167]:
res['n_calls_answer_queue']=df[(df.state_desc=='Answer') & (df.call_type=='queue')].groupby('call_date').id.nunique()
res['n_calls_answer_ivr']=df[(df.state_desc=='Answer') & (df.call_type=='ivr')].groupby('call_date').id.nunique()


# dispositions
res['n_calls_with_disposition1']=df[(df.state_desc=='Answer')&(df.disposition.notna())].groupby('call_date').id.nunique()
res['n_calls_with_disposition2']=df[(df.state_desc=='Answer')&(df.disposition_2.notna())].groupby('call_date').id.nunique()
res['n_calls_with_disposition3']=df[(df.state_desc=='Answer')&(df.disposition_3.notna())].groupby('call_date').id.nunique()

res['perc_calls_with_disposition1']=res.n_calls_with_disposition1/res.n_calls_answer_queue*100
res['perc_calls_with_disposition2']=res.n_calls_with_disposition2/res.n_calls_answer_queue*100
res['perc_calls_with_disposition3']=res.n_calls_with_disposition3/res.n_calls_answer_queue*100

res['perc_calls_with_disposition1_reschedule']=df[(df.state_desc=='Answer')&((df.disposition=='reschedule')|(df.disposition=='recallme'))].groupby('call_date').id.nunique()/res.n_calls_answer_queue*100
res['perc_calls_with_disposition2_PTP']=df[(df.state_desc=='Answer')&(df.disposition_2=='Promise to pay')].groupby('call_date').id.nunique()/res.n_calls_answer_queue*100



In [168]:
res=res.drop(columns=['n_calls_with_disposition1','n_calls_with_disposition2','n_calls_with_disposition3','n_calls_answer_queue','n_calls_answer_ivr'])

In [169]:
res=res.fillna(0)

In [170]:
res.index=res.index.astype('str')

In [171]:
res.head(2)

,n_calls,n_queue_calls,n_ivr_calls,day_of_week,avg_calls_per_hour,avg_ivr_calls_per_hour,avg_queue_calls_per_hour,n_calls_ln0,n_calls_ln0_queue,n_calls_ln0_ivr,n_calls_legal,n_ivr_calls_legal,n_queue_calls_legal,n_calls_mid_priority,n_ivr_calls_mid_priority,n_queue_calls_mid_priority,n_calls_initial,n_ivr_calls_initial,n_queue_calls_initial,n_calls_low_priority,n_ivr_calls_low_priority,n_queue_calls_low_priority,n_calls_high_priority,n_ivr_calls_high_priority,n_queue_calls_high_priority,n_calls_mixed_priority,n_ivr_calls_mixed_priority,n_queue_calls_mixed_priority,perc_calls_busy,perc_calls_no_answer,perc_calls_congestion,perc_calls_answer,perc_calls_abandoned,perc_calls_global,perc_calls_no_such_number,perc_calls_drop,perc_calls_answer_queue,perc_calls_answer_ivr,perc_calls_with_disposition1,perc_calls_with_disposition2,perc_calls_with_disposition3,perc_calls_with_disposition1_reschedule,perc_calls_with_disposition2_PTP
call_date,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2024-09-28,72468,24163.0,48305,Sat,5176.285714,3450.357143,2013.583333,18848,4366.0,14482,7880.0,0.0,7880.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,41.695645,32.887619,0.118673,17.607772,1.404758,0.216647,0.516090,0.211128,16.020362,18.401822,1.834151,1.808318,1.808318,0.387497,0.981658
2024-09-29,24317,0.0,24317,Sun,3473.857143,3473.857143,0.000000,3228,0.0,3228,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,27.807707,34.514948,0.106921,22.757741,0.000000,0.000000,0.394786,0.000000,0.000000,22.757741,0.000000,0.000000,0.000000,0.000000,0.000000


In [172]:
res.shape

(64, 43)

In [173]:
res=res.join(add2)

In [174]:
res=res.join(add3)

In [175]:
res.n_manual_outbound_calls=res.n_manual_outbound_calls.fillna(0)
res.n_inbound_calls=res.n_inbound_calls.fillna(0)
res.n_sms_sent=res.n_sms_sent.fillna(0)

In [176]:
res.tail(2)

,n_calls,n_queue_calls,n_ivr_calls,day_of_week,avg_calls_per_hour,avg_ivr_calls_per_hour,avg_queue_calls_per_hour,n_calls_ln0,n_calls_ln0_queue,n_calls_ln0_ivr,n_calls_legal,n_ivr_calls_legal,n_queue_calls_legal,n_calls_mid_priority,n_ivr_calls_mid_priority,n_queue_calls_mid_priority,n_calls_initial,n_ivr_calls_initial,n_queue_calls_initial,n_calls_low_priority,n_ivr_calls_low_priority,n_queue_calls_low_priority,n_calls_high_priority,n_ivr_calls_high_priority,n_queue_calls_high_priority,n_calls_mixed_priority,n_ivr_calls_mixed_priority,n_queue_calls_mixed_priority,perc_calls_busy,perc_calls_no_answer,perc_calls_congestion,perc_calls_answer,perc_calls_abandoned,perc_calls_global,perc_calls_no_such_number,perc_calls_drop,perc_calls_answer_queue,perc_calls_answer_ivr,perc_calls_with_disposition1,perc_calls_with_disposition2,perc_calls_with_disposition3,perc_calls_with_disposition1_reschedule,perc_calls_with_disposition2_PTP,n_manual_outbound_calls,n_inbound_calls,n_sms_sent,n_sms_sent_dpd_m3,n_sms_sent_dpd_m1,n_sms_sent_dpd_0,n_sms_sent_dpd_1,n_sms_sent_dpd_3,n_sms_sent_dpd_7,n_sms_sent_dpd_18,n_sms_sent_dpd_25
call_date,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2024-11-29,131151,52173.0,78978,Fri,9367.928571,5641.285714,4013.307692,16847,11086.0,5761,14343.0,0.0,14343.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,44.000427,29.003973,0.082348,19.773391,2.612256,0.0,0.212732,0.366753,11.082361,25.514700,0.466966,0.466966,0.466966,0.103770,0.207541,185.0,0.0,185442,4322.0,0.0,1484.0,657.0,610.0,127.0,208.0,487.0
2024-11-30,91532,31694.0,59838,Sat,6538.000000,4274.142857,2641.166667,23264,5846.0,17418,8214.0,0.0,8214.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,42.565442,37.758380,0.041516,11.824280,2.522615,0.0,0.695931,0.327754,11.655203,11.913834,0.216567,0.216567,0.216567,0.027071,0.108284,21.0,0.0,158287,6046.0,0.0,0.0,832.0,1678.0,0.0,85.0,225.0


In [177]:
call_date_sets = xc_data[xc_data.start_time.notna()].groupby('call_date')['loan_id'].apply(set).reset_index()
call_date_sets.call_date=call_date_sets.call_date.astype('str')

In [178]:
lb_hist_dpd0_2_set = lb_hist[lb_hist.days_late.between(0,2)].groupby('period')['loan_id'].apply(set).reset_index()
lb_hist_dpd3_15_set = lb_hist[lb_hist.days_late.between(3,15)].groupby('period')['loan_id'].apply(set).reset_index()
lb_hist_dpd16_30_set = lb_hist[lb_hist.days_late.between(16,30)].groupby('period')['loan_id'].apply(set).reset_index()


In [179]:
# exclusions_dpd0_set=xc_data[xc_data.is_exclusion_DPD_0_M1_M3_M5==1].groupby('call_date')['loan_id'].apply(set).reset_index()
exclusions_dpd3_15_set=xc_data[xc_data.is_exclusion_DPD_3_15==1].groupby('call_date')['loan_id'].apply(set).reset_index()
exclusions_dpd16_30_set=xc_data[xc_data.is_exclusion_DPD_16_30==1].groupby('call_date')['loan_id'].apply(set).reset_index()




In [182]:
# exclusions_dpd3_15_set.call_date=exclusions_dpd3_15_set.call_date.astype('str')
exclusions_dpd3_15_set.call_date=pd.to_datetime(exclusions_dpd3_15_set.call_date)

In [183]:
add4 = pd.merge(lb_hist_dpd3_15_set, exclusions_dpd3_15_set, left_on='period', right_on='call_date')

# Subtract exclusions from lb_hist sets
add4['n_loans_expected_calls_dpd3_15'] = add4.apply(
    lambda row: len(row['loan_id_x'] - row['loan_id_y']),
    axis=1
)

In [184]:
add4['set_loans_expected_calls_dpd3_15']=add4.loan_id_x-add4.loan_id_y

In [185]:
add4['n_loans_expected_calls_dpd3_15']=add4['n_loans_expected_calls_dpd3_15']

In [186]:
call_date_sets.call_date=pd.to_datetime(call_date_sets.call_date)

In [187]:
add5_1 = pd.merge(add4, call_date_sets, left_on='period', right_on='call_date')

# Create 'loans_called_to' column: intersection of 'n_loans_expected_calls_dpd3_15' and 'loan_id'
add5_1['loans_called_to'] = add5_1.apply(
    lambda row: row['set_loans_expected_calls_dpd3_15']&(row['loan_id']),
    axis=1
)

# Create 'loans_expected_not_called' column: difference of 'n_loans_expected_calls_dpd3_15' and 'loan_id'
add5_1['loans_expected_not_called'] = add5_1.set_loans_expected_calls_dpd3_15-add5_1.loans_called_to
add5_1['n_loans_lb_hist'] = add5_1['loan_id_x'].apply(len)
add5_1['n_loans_excluded'] = add5_1['loan_id_y'].apply(len)
add5_1['n_loans_called_dpd3_15'] = add5_1['loans_called_to'].apply(len)


In [188]:
res.shape

(64, 54)

In [190]:
add5_1.period=add5_1.period.astype('str')

In [191]:
res=res.join(add5_1[['period','n_loans_expected_calls_dpd3_15','loans_expected_not_called','n_loans_called_dpd3_15']].set_index('period'))
res.head(2)

,n_calls,n_queue_calls,n_ivr_calls,day_of_week,avg_calls_per_hour,avg_ivr_calls_per_hour,avg_queue_calls_per_hour,n_calls_ln0,n_calls_ln0_queue,n_calls_ln0_ivr,n_calls_legal,n_ivr_calls_legal,n_queue_calls_legal,n_calls_mid_priority,n_ivr_calls_mid_priority,n_queue_calls_mid_priority,n_calls_initial,n_ivr_calls_initial,n_queue_calls_initial,n_calls_low_priority,n_ivr_calls_low_priority,n_queue_calls_low_priority,n_calls_high_priority,n_ivr_calls_high_priority,n_queue_calls_high_priority,n_calls_mixed_priority,n_ivr_calls_mixed_priority,n_queue_calls_mixed_priority,perc_calls_busy,perc_calls_no_answer,perc_calls_congestion,perc_calls_answer,perc_calls_abandoned,perc_calls_global,perc_calls_no_such_number,perc_calls_drop,perc_calls_answer_queue,perc_calls_answer_ivr,perc_calls_with_disposition1,perc_calls_with_disposition2,perc_calls_with_disposition3,perc_calls_with_disposition1_reschedule,perc_calls_with_disposition2_PTP,n_manual_outbound_calls,n_inbound_calls,n_sms_sent,n_sms_sent_dpd_m3,n_sms_sent_dpd_m1,n_sms_sent_dpd_0,n_sms_sent_dpd_1,n_sms_sent_dpd_3,n_sms_sent_dpd_7,n_sms_sent_dpd_18,n_sms_sent_dpd_25,n_loans_expected_calls_dpd3_15,loans_expected_not_called,n_loans_called_dpd3_15
call_date,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2024-09-28,72468,24163.0,48305,Sat,5176.285714,3450.357143,2013.583333,18848,4366.0,14482,7880.0,0.0,7880.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,41.695645,32.887619,0.118673,17.607772,1.404758,0.216647,0.516090,0.211128,16.020362,18.401822,1.834151,1.808318,1.808318,0.387497,0.981658,41.0,0.0,105794,1410.0,0.0,0.0,2194.0,691.0,0.0,38.0,232.0,NaN,NaN,NaN
2024-09-29,24317,0.0,24317,Sun,3473.857143,3473.857143,0.000000,3228,0.0,3228,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,27.807707,34.514948,0.106921,22.757741,0.000000,0.000000,0.394786,0.000000,0.000000,22.757741,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,216036,10414.0,17116.0,0.0,0.0,1763.0,0.0,55.0,40.0,4713.0,"{TIXJ914, EOND935, GRUU807, MENC530, YGGY757, ...",1882.0


In [192]:
res['perc_loans_called_dpd3_15']=res.n_loans_called_dpd3_15/res.n_loans_expected_calls_dpd3_15*100

In [193]:
res=res.rename(columns={'loans_expected_not_called':'loans_expected_not_called_dpd3_15'})

In [195]:
exclusions_dpd16_30_set.call_date=pd.to_datetime(exclusions_dpd16_30_set.call_date)

In [196]:
add4 = pd.merge(lb_hist_dpd16_30_set, exclusions_dpd16_30_set, left_on='period', right_on='call_date')

# Subtract exclusions from lb_hist sets
add4['n_loans_expected_calls_dpd16_30'] = add4.apply(
    lambda row: len(row['loan_id_x'] - row['loan_id_y']),
    axis=1
)

In [197]:
add4['set_loans_expected_calls_dpd16_30']=add4.loan_id_x-add4.loan_id_y

In [198]:
add4['n_loans_expected_calls_dpd16_30']=add4['n_loans_expected_calls_dpd16_30']

In [199]:
add5_2 = pd.merge(add4, call_date_sets, left_on='period', right_on='call_date')

# Create 'loans_called_to' column: intersection of 'n_loans_expected_calls_dpd3_15' and 'loan_id'
add5_2['loans_called_to'] = add5_2.apply(
    lambda row: row['set_loans_expected_calls_dpd16_30']&(row['loan_id']),
    axis=1
)

# Create 'loans_expected_not_called' column: difference of 'n_loans_expected_calls_dpd3_15' and 'loan_id'
add5_2['loans_expected_not_called'] = add5_2.set_loans_expected_calls_dpd16_30-add5_2.loans_called_to
add5_2['n_loans_lb_hist'] = add5_2['loan_id_x'].apply(len)
add5_2['n_loans_excluded'] = add5_2['loan_id_y'].apply(len)
add5_2['n_loans_called_dpd16_30'] = add5_2['loans_called_to'].apply(len)


In [200]:
add5_2.period=add5_2.period.astype('str')

In [201]:
res=res.join(add5_2[['period','n_loans_expected_calls_dpd16_30','loans_expected_not_called','n_loans_called_dpd16_30']].set_index('period'))
res.head(2)

,n_calls,n_queue_calls,n_ivr_calls,day_of_week,avg_calls_per_hour,avg_ivr_calls_per_hour,avg_queue_calls_per_hour,n_calls_ln0,n_calls_ln0_queue,n_calls_ln0_ivr,n_calls_legal,n_ivr_calls_legal,n_queue_calls_legal,n_calls_mid_priority,n_ivr_calls_mid_priority,n_queue_calls_mid_priority,n_calls_initial,n_ivr_calls_initial,n_queue_calls_initial,n_calls_low_priority,n_ivr_calls_low_priority,n_queue_calls_low_priority,n_calls_high_priority,n_ivr_calls_high_priority,n_queue_calls_high_priority,n_calls_mixed_priority,n_ivr_calls_mixed_priority,n_queue_calls_mixed_priority,perc_calls_busy,perc_calls_no_answer,perc_calls_congestion,perc_calls_answer,perc_calls_abandoned,perc_calls_global,perc_calls_no_such_number,perc_calls_drop,perc_calls_answer_queue,perc_calls_answer_ivr,perc_calls_with_disposition1,perc_calls_with_disposition2,perc_calls_with_disposition3,perc_calls_with_disposition1_reschedule,perc_calls_with_disposition2_PTP,n_manual_outbound_calls,n_inbound_calls,n_sms_sent,n_sms_sent_dpd_m3,n_sms_sent_dpd_m1,n_sms_sent_dpd_0,n_sms_sent_dpd_1,n_sms_sent_dpd_3,n_sms_sent_dpd_7,n_sms_sent_dpd_18,n_sms_sent_dpd_25,n_loans_expected_calls_dpd3_15,loans_expected_not_called_dpd3_15,n_loans_called_dpd3_15,perc_loans_called_dpd3_15,n_loans_expected_calls_dpd16_30,loans_expected_not_called,n_loans_called_dpd16_30
call_date,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2024-09-28,72468,24163.0,48305,Sat,5176.285714,3450.357143,2013.583333,18848,4366.0,14482,7880.0,0.0,7880.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,41.695645,32.887619,0.118673,17.607772,1.404758,0.216647,0.516090,0.211128,16.020362,18.401822,1.834151,1.808318,1.808318,0.387497,0.981658,41.0,0.0,105794,1410.0,0.0,0.0,2194.0,691.0,0.0,38.0,232.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2024-09-29,24317,0.0,24317,Sun,3473.857143,3473.857143,0.000000,3228,0.0,3228,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,27.807707,34.514948,0.106921,22.757741,0.000000,0.000000,0.394786,0.000000,0.000000,22.757741,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,216036,10414.0,17116.0,0.0,0.0,1763.0,0.0,55.0,40.0,4713.0,"{TIXJ914, EOND935, GRUU807, MENC530, YGGY757, ...",1882.0,39.932103,7105.0,"{LSER718, QTSX035, AKWI351, FBKA151, UNUI561, ...",1008.0


In [202]:
res=res.rename(columns={'loans_expected_not_called':'loans_expected_not_called_dpd16_30'})

In [203]:
res['perc_loans_called_dpd16_30']=res.n_loans_called_dpd16_30/res.n_loans_expected_calls_dpd16_30*100

In [205]:
add5_3 = pd.merge(lb_hist_dpd0_2_set, call_date_sets, left_on='period', right_on='call_date').rename(columns={'loan_id_x':'set_loans_expected_calls','loan_id_y':'set_loans_called_to'})
# add5

In [206]:
# Create 'loans_called_to' column: intersection of 'n_loans_expected_calls_dpd3_15' and 'loan_id'
add5_3['loans_called_to'] = add5_3.apply(
    lambda row: row['set_loans_expected_calls']&(row['set_loans_called_to']),
    axis=1
)

# Create 'loans_expected_not_called' column: difference of 'n_loans_expected_calls_dpd3_15' and 'loan_id'
add5_3['n_loans_expected_calls_dpd0_2']=add5_3['set_loans_expected_calls'].apply(len)
add5_3['loans_expected_not_called'] = add5_3.set_loans_expected_calls-add5_3.loans_called_to
add5_3['n_loans_called_dpd0_2'] = add5_3['loans_called_to'].apply(len)

add5_3.head(20)

,period,set_loans_expected_calls,call_date,set_loans_called_to,loans_called_to,n_loans_expected_calls_dpd0_2,loans_expected_not_called,n_loans_called_dpd0_2
0,2024-09-28,"{JQPO403, QBIR535, MKLV076, ZPDR600, LWSK934, ...",2024-09-28,"{JQPO403, FHQO043, CSDB172, MPKW600, AGFO458, ...","{JQPO403, QBIR535, MKLV076, LWSK934, ZPDR600, ...",4329,"{XDYA264, MMNR147, MFVC680, KLPG962, UAJJ865, ...",3225
1,2024-09-29,"{JQPO403, MKLV076, ZPDR600, MFVC680, KLPG962, ...",2024-09-29,"{JQPO403, UAJJ865, FHQO043, LIXY730, MPKW600, ...","{JQPO403, MKLV076, MFVC680, KLPG962, RQGY816, ...",2398,"{QJWG229, QUYC970, ZPDR600, REHC889, FEYI725, ...",2220
2,2024-09-30,"{BXPR365, IEKZ361, PAGN796, IOUG579, HABM684, ...",2024-09-30,"{JQPO403, ENOH219, FHQO043, LIXY730, FBKA151, ...","{BXPR365, IOUG579, KLFX107, IFEW985, TNQU635, ...",3981,"{HABM684, IEKZ361, PAGN796, EQIB953, TAIS168, ...",2385
3,2024-10-01,"{YTLQ291, TAIS168, ASNZ635, VQPT342, GHGO064, ...",2024-10-01,"{JQPO403, FHQO043, LIXY730, FBKA151, PIIX355, ...","{BXPR365, IGZA570, KLFX107, CGVQ210, YTLQ291, ...",5001,"{IOUG579, OUAZ761, AUYM633, HLUL898, NCQW865, ...",2897
4,2024-10-02,"{TAIS168, VQPT342, GHGO064, BMGC847, QVWI796, ...",2024-10-02,"{JQPO403, LIXY730, FBKA151, PIIX355, AGFO458, ...","{TAIS168, GHGO064, BMGC847, QVWI796, MBKD297, ...",6312,"{FHUA821, VQPT342, GREP070, PRUF680, FCGO241, ...",5943
5,2024-10-03,"{PIIX355, RJAP846, VQPT342, IMEH721, FKUD271, ...",2024-10-03,"{JQPO403, LIXY730, FBKA151, PIIX355, AGFO458, ...","{PIIX355, RJAP846, VQPT342, IMEH721, FKUD271, ...",5628,"{LDFV275, AMJP769, FOAN736, JXKE740, QIUD631, ...",5204
6,2024-10-04,"{CCXN171, BSFW541, ODQZ013, ZMRH808, PIIX355, ...",2024-10-04,"{JQPO403, KHST260, LIXY730, CSDB172, FBKA151, ...","{CCXN171, BSFW541, ODQZ013, ZMRH808, PIIX355, ...",4588,"{HSYW963, AJPF611, AYSL965, FVKM850, RSKF614, ...",4301
7,2024-10-05,"{CCXN171, ZMRH808, PIIX355, CMUY814, DVVK920, ...",2024-10-05,"{JQPO403, LIXY730, FBKA151, PIIX355, AGFO458, ...","{CCXN171, ZMRH808, CMUY814, PIIX355, NFPD003, ...",2797,"{QKPE045, SLJL818, CMVF231, UTOT935, DVVK920, ...",2678
8,2024-10-06,"{SUVM063, GKMS184, GVRT458, ZMRH808, NVYU306, ...",2024-10-06,"{YIZC914, PIIX355, EEZO679, PXYV390, XIVL497, ...","{SUVM063, GKMS184, GVRT458, ZMRH808, NVYU306, ...",763,{XSXI640},762
9,2024-10-07,"{GOXF031, ZKUK188, WEGJ958, FCFE700, HIJM722, ...",2024-10-07,"{JQPO403, LIXY730, FBKA151, PIIX355, IYEW273, ...","{CNTA749, MMWE942, PHWZ337, CKTE200, ZVLE768, ...",1418,"{GOXF031, NQYS670, ESHC601, NWWV687, QWCS527, ...",680


In [207]:
add5_3.period=add5_3.period.astype('str')

In [208]:
res=res.join(add5_3[['period','n_loans_expected_calls_dpd0_2','loans_expected_not_called','n_loans_called_dpd0_2']].set_index('period'))
res.head(2)

,n_calls,n_queue_calls,n_ivr_calls,day_of_week,avg_calls_per_hour,avg_ivr_calls_per_hour,avg_queue_calls_per_hour,n_calls_ln0,n_calls_ln0_queue,n_calls_ln0_ivr,n_calls_legal,n_ivr_calls_legal,n_queue_calls_legal,n_calls_mid_priority,n_ivr_calls_mid_priority,n_queue_calls_mid_priority,n_calls_initial,n_ivr_calls_initial,n_queue_calls_initial,n_calls_low_priority,n_ivr_calls_low_priority,n_queue_calls_low_priority,n_calls_high_priority,n_ivr_calls_high_priority,n_queue_calls_high_priority,n_calls_mixed_priority,n_ivr_calls_mixed_priority,n_queue_calls_mixed_priority,perc_calls_busy,perc_calls_no_answer,perc_calls_congestion,perc_calls_answer,perc_calls_abandoned,perc_calls_global,perc_calls_no_such_number,perc_calls_drop,perc_calls_answer_queue,perc_calls_answer_ivr,perc_calls_with_disposition1,perc_calls_with_disposition2,perc_calls_with_disposition3,perc_calls_with_disposition1_reschedule,perc_calls_with_disposition2_PTP,n_manual_outbound_calls,n_inbound_calls,n_sms_sent,n_sms_sent_dpd_m3,n_sms_sent_dpd_m1,n_sms_sent_dpd_0,n_sms_sent_dpd_1,n_sms_sent_dpd_3,n_sms_sent_dpd_7,n_sms_sent_dpd_18,n_sms_sent_dpd_25,n_loans_expected_calls_dpd3_15,loans_expected_not_called_dpd3_15,n_loans_called_dpd3_15,perc_loans_called_dpd3_15,n_loans_expected_calls_dpd16_30,loans_expected_not_called_dpd16_30,n_loans_called_dpd16_30,perc_loans_called_dpd16_30,n_loans_expected_calls_dpd0_2,loans_expected_not_called,n_loans_called_dpd0_2
call_date,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2024-09-28,72468,24163.0,48305,Sat,5176.285714,3450.357143,2013.583333,18848,4366.0,14482,7880.0,0.0,7880.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,41.695645,32.887619,0.118673,17.607772,1.404758,0.216647,0.516090,0.211128,16.020362,18.401822,1.834151,1.808318,1.808318,0.387497,0.981658,41.0,0.0,105794,1410.0,0.0,0.0,2194.0,691.0,0.0,38.0,232.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4329,"{XDYA264, MMNR147, MFVC680, KLPG962, UAJJ865, ...",3225
2024-09-29,24317,0.0,24317,Sun,3473.857143,3473.857143,0.000000,3228,0.0,3228,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,27.807707,34.514948,0.106921,22.757741,0.000000,0.000000,0.394786,0.000000,0.000000,22.757741,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,216036,10414.0,17116.0,0.0,0.0,1763.0,0.0,55.0,40.0,4713.0,"{TIXJ914, EOND935, GRUU807, MENC530, YGGY757, ...",1882.0,39.932103,7105.0,"{LSER718, QTSX035, AKWI351, FBKA151, UNUI561, ...",1008.0,14.187192,2398,"{QJWG229, QUYC970, ZPDR600, REHC889, FEYI725, ...",2220


In [209]:
res=res.rename(columns={'loans_expected_not_called':'loans_expected_not_called_dpd0_2'})

In [210]:
res['perc_loans_called_dpd0_2']=res.n_loans_called_dpd0_2/res.n_loans_expected_calls_dpd0_2*100

In [211]:
# res=res.iloc[:-1,:]

In [212]:
res

,n_calls,n_queue_calls,n_ivr_calls,day_of_week,avg_calls_per_hour,avg_ivr_calls_per_hour,avg_queue_calls_per_hour,n_calls_ln0,n_calls_ln0_queue,n_calls_ln0_ivr,n_calls_legal,n_ivr_calls_legal,n_queue_calls_legal,n_calls_mid_priority,n_ivr_calls_mid_priority,n_queue_calls_mid_priority,n_calls_initial,n_ivr_calls_initial,n_queue_calls_initial,n_calls_low_priority,n_ivr_calls_low_priority,n_queue_calls_low_priority,n_calls_high_priority,n_ivr_calls_high_priority,n_queue_calls_high_priority,n_calls_mixed_priority,n_ivr_calls_mixed_priority,n_queue_calls_mixed_priority,perc_calls_busy,perc_calls_no_answer,perc_calls_congestion,perc_calls_answer,perc_calls_abandoned,perc_calls_global,perc_calls_no_such_number,perc_calls_drop,perc_calls_answer_queue,perc_calls_answer_ivr,perc_calls_with_disposition1,perc_calls_with_disposition2,perc_calls_with_disposition3,perc_calls_with_disposition1_reschedule,perc_calls_with_disposition2_PTP,n_manual_outbound_calls,n_inbound_calls,n_sms_sent,n_sms_sent_dpd_m3,n_sms_sent_dpd_m1,n_sms_sent_dpd_0,n_sms_sent_dpd_1,n_sms_sent_dpd_3,n_sms_sent_dpd_7,n_sms_sent_dpd_18,n_sms_sent_dpd_25,n_loans_expected_calls_dpd3_15,loans_expected_not_called_dpd3_15,n_loans_called_dpd3_15,perc_loans_called_dpd3_15,n_loans_expected_calls_dpd16_30,loans_expected_not_called_dpd16_30,n_loans_called_dpd16_30,perc_loans_called_dpd16_30,n_loans_expected_calls_dpd0_2,loans_expected_not_called_dpd0_2,n_loans_called_dpd0_2,perc_loans_called_dpd0_2
call_date,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2024-09-28,72468,24163.0,48305,Sat,5176.285714,3450.357143,2013.583333,18848,4366.0,14482,7880.0,0.0,7880.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,41.695645,32.887619,0.118673,17.607772,1.404758,0.216647,0.516090,0.211128,16.020362,18.401822,1.834151,1.808318,1.808318,0.387497,0.981658,41.0,0.0,105794,1410.0,0.0,0.0,2194.0,691.0,0.0,38.0,232.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4329,"{XDYA264, MMNR147, MFVC680, KLPG962, UAJJ865, ...",3225,74.497574
2024-09-29,24317,0.0,24317,Sun,3473.857143,3473.857143,0.000000,3228,0.0,3228,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,27.807707,34.514948,0.106921,22.757741,0.000000,0.000000,0.394786,0.000000,0.000000,22.757741,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,216036,10414.0,17116.0,0.0,0.0,1763.0,0.0,55.0,40.0,4713.0,"{TIXJ914, EOND935, GRUU807, MENC530, YGGY757, ...",1882.0,39.932103,7105.0,"{LSER718, QTSX035, AKWI351, FBKA151, UNUI561, ...",1008.0,14.187192,2398,"{QJWG229, QUYC970, ZPDR600, REHC889, FEYI725, ...",2220,92.577148
2024-09-30,68995,39519.0,29476,Mon,4928.214286,2105.428571,3039.923077,17793,6753.0,11040,9171.0,0.0,9171.0,8813.0,0.0,8813.0,4897.0,0.0,4897.0,4395.0,0.0,4395.0,0.0,0.0,0.0,7723.0,0.0,7723.0,41.574027,33.985071,0.156533,16.035945,3.137909,0.000000,0.311617,0.481194,15.367292,16.932420,0.000000,0.000000,0.000000,0.000000,0.000000,62.0,0.0,134168,2706.0,4992.0,1254.0,0.0,639.0,0.0,54.0,81.0,NaN,NaN,NaN,NaN,5723.0,"{RPLC224, YWZD963, PSDR150, OASC872, WKTL823, ...",5253.0,91.787524,3981,"{HABM684, IEKZ361, PAGN796, EQIB953, TAIS168, ...",2385,59.909570
2024-10-01,108847,42129.0,66718,Tue,7774.785714,4765.571429,3009.214286,25607,8065.0,17542,10248.0,0.0,10248.0,11820.0,0.0,11820.0,5265.0,0.0,5265.0,6254.0,0.0,6254.0,0.0,0.0,0.0,3759.0,0.0,3759.0,48.375242,29.098643,0.194769,14.520382,1.700552,0.001837,0.298584,0.182825,12.708586,15.664438,0.074710,0.074710,0.074710,0.037355,0.037355,49.0,0.0,141521,4129.0,11205.0,457.0,800.0,0.0,288.0,126.0,90.0,5201.0,"{QSZC433, LGXC182, WEYL321, SFAC000, ZSFW368, ...",5113.0,98.308018,5153.0,"{UEEF073, DXAU816, VHVE316, LASB551, DKEB465, ...",5025.0,97.516010,5001,"{IOUG579, OUAZ761, AUYM633, HLUL898, NCQW865, ...",2897,57.928414
2024-10-02,118010,40781.0,77229,Wed,8429.285714,5516.357143,2912.928571,30329,7692.0,22637,9255.0,0.0,9255.0,11376.0,0.0,11376.0,4855.0,0.0,4855.0,5645.0,0.0,5645.0,0.0,0.0,0.0,5582.0,0.0,5582.0,44.942801,31.427845,0.203373,14.502

### Upload to SF

In [300]:
res_final=res.iloc[3:,:]

In [306]:
res_final.head(2)

,n_calls,n_queue_calls,n_ivr_calls,day_of_week,avg_calls_per_hour,avg_ivr_calls_per_hour,avg_queue_calls_per_hour,n_calls_ln0,n_calls_ln0_queue,n_calls_ln0_ivr,n_calls_legal,n_ivr_calls_legal,n_queue_calls_legal,n_calls_mid_priority,n_ivr_calls_mid_priority,n_queue_calls_mid_priority,n_calls_initial,n_ivr_calls_initial,n_queue_calls_initial,n_calls_low_priority,n_ivr_calls_low_priority,n_queue_calls_low_priority,n_calls_high_priority,n_ivr_calls_high_priority,n_queue_calls_high_priority,n_calls_mixed_priority,n_ivr_calls_mixed_priority,n_queue_calls_mixed_priority,perc_calls_busy,perc_calls_no_answer,perc_calls_congestion,perc_calls_answer,perc_calls_abandoned,perc_calls_global,perc_calls_no_such_number,perc_calls_drop,perc_calls_answer_queue,perc_calls_answer_ivr,perc_calls_with_disposition1,perc_calls_with_disposition2,perc_calls_with_disposition3,perc_calls_with_disposition1_reschedule,perc_calls_with_disposition2_PTP,n_manual_outbound_calls,n_inbound_calls,n_sms_sent,n_sms_sent_dpd_m3,n_sms_sent_dpd_m1,n_sms_sent_dpd_0,n_sms_sent_dpd_1,n_sms_sent_dpd_3,n_sms_sent_dpd_7,n_sms_sent_dpd_18,n_sms_sent_dpd_25,n_loans_expected_calls_dpd3_15,loans_expected_not_called_dpd3_15,n_loans_called_dpd3_15,perc_loans_called_dpd3_15,n_loans_expected_calls_dpd16_30,loans_expected_not_called_dpd16_30,n_loans_called_dpd16_30,perc_loans_called_dpd16_30,n_loans_expected_calls_dpd0_2,loans_expected_not_called_dpd0_2,n_loans_called_dpd0_2,perc_loans_called_dpd0_2
call_date,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2024-10-01,108847,42129.0,66718,Tue,7774.785714,4765.571429,3009.214286,25607,8065.0,17542,10248.0,0.0,10248.0,11820.0,0.0,11820.0,5265.0,0.0,5265.0,6254.0,0.0,6254.0,0.0,0.0,0.0,3759.0,0.0,3759.0,48.375242,29.098643,0.194769,14.520382,1.700552,0.001837,0.298584,0.182825,12.708586,15.664438,0.074710,0.074710,0.074710,0.037355,0.037355,49.0,0.0,141521,4129.0,11205.0,457.0,800.0,0.0,288.0,126.0,90.0,5201.0,"{QSZC433, LGXC182, WEYL321, SFAC000, ZSFW368, ...",5113.0,98.308018,5153.0,"{UEEF073, DXAU816, VHVE316, LASB551, DKEB465, ...",5025.0,97.516010,5001,"{IOUG579, OUAZ761, AUYM633, HLUL898, NCQW865, ...",2897,57.928414
2024-10-02,118010,40781.0,77229,Wed,8429.285714,5516.357143,2912.928571,30329,7692.0,22637,9255.0,0.0,9255.0,11376.0,0.0,11376.0,4855.0,0.0,4855.0,5645.0,0.0,5645.0,0.0,0.0,0.0,5582.0,0.0,5582.0,44.942801,31.427845,0.203373,14.502161,1.504110,0.000000,0.384713,0.190662,14.514112,14.495850,0.219632,0.219632,0.219632,0.050684,0.084474,30.0,0.0,139173,0.0,12033.0,714.0,377.0,0.0,132.0,0.0,0.0,4576.0,"{BBBU890, BRKI251, UTJY519, VUNL729, FPLB522}",4571.0,99.890734,5077.0,"{KPEI715, CCUF236, ISSX669, MJGD070, QYHB461, ...",5007.0,98.621233,6312,"{FHUA821, VQPT342, GREP070, PRUF680, FCGO241, ...",5943,94.153992


In [308]:
res_final['loans_expected_not_called_dpd3_15'] = res_final['loans_expected_not_called_dpd3_15'].apply(
    lambda x: np.nan if pd.isna(x) or not x else (', '.join(map(str, x)) if isinstance(x, set) else x)
)

In [310]:
res_final['loans_expected_not_called_dpd0_2'] = res_final['loans_expected_not_called_dpd0_2'].apply(
    lambda x: np.nan if pd.isna(x) or not x else (', '.join(map(str, x)) if isinstance(x, set) else x)
)

In [312]:
res_final['loans_expected_not_called_dpd16_30'] = res_final['loans_expected_not_called_dpd16_30'].apply(
    lambda x: np.nan if pd.isna(x) or not x else (', '.join(map(str, x)) if isinstance(x, set) else x)
)

In [314]:
res_final.head(2)

,n_calls,n_queue_calls,n_ivr_calls,day_of_week,avg_calls_per_hour,avg_ivr_calls_per_hour,avg_queue_calls_per_hour,n_calls_ln0,n_calls_ln0_queue,n_calls_ln0_ivr,n_calls_legal,n_ivr_calls_legal,n_queue_calls_legal,n_calls_mid_priority,n_ivr_calls_mid_priority,n_queue_calls_mid_priority,n_calls_initial,n_ivr_calls_initial,n_queue_calls_initial,n_calls_low_priority,n_ivr_calls_low_priority,n_queue_calls_low_priority,n_calls_high_priority,n_ivr_calls_high_priority,n_queue_calls_high_priority,n_calls_mixed_priority,n_ivr_calls_mixed_priority,n_queue_calls_mixed_priority,perc_calls_busy,perc_calls_no_answer,perc_calls_congestion,perc_calls_answer,perc_calls_abandoned,perc_calls_global,perc_calls_no_such_number,perc_calls_drop,perc_calls_answer_queue,perc_calls_answer_ivr,perc_calls_with_disposition1,perc_calls_with_disposition2,perc_calls_with_disposition3,perc_calls_with_disposition1_reschedule,perc_calls_with_disposition2_PTP,n_manual_outbound_calls,n_inbound_calls,n_sms_sent,n_sms_sent_dpd_m3,n_sms_sent_dpd_m1,n_sms_sent_dpd_0,n_sms_sent_dpd_1,n_sms_sent_dpd_3,n_sms_sent_dpd_7,n_sms_sent_dpd_18,n_sms_sent_dpd_25,n_loans_expected_calls_dpd3_15,loans_expected_not_called_dpd3_15,n_loans_called_dpd3_15,perc_loans_called_dpd3_15,n_loans_expected_calls_dpd16_30,loans_expected_not_called_dpd16_30,n_loans_called_dpd16_30,perc_loans_called_dpd16_30,n_loans_expected_calls_dpd0_2,loans_expected_not_called_dpd0_2,n_loans_called_dpd0_2,perc_loans_called_dpd0_2
call_date,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2024-10-01,108847,42129.0,66718,Tue,7774.785714,4765.571429,3009.214286,25607,8065.0,17542,10248.0,0.0,10248.0,11820.0,0.0,11820.0,5265.0,0.0,5265.0,6254.0,0.0,6254.0,0.0,0.0,0.0,3759.0,0.0,3759.0,48.375242,29.098643,0.194769,14.520382,1.700552,0.001837,0.298584,0.182825,12.708586,15.664438,0.074710,0.074710,0.074710,0.037355,0.037355,49.0,0.0,141521,4129.0,11205.0,457.0,800.0,0.0,288.0,126.0,90.0,5201.0,"QSZC433, LGXC182, WEYL321, SFAC000, ZSFW368, F...",5113.0,98.308018,5153.0,"UEEF073, DXAU816, VHVE316, LASB551, DKEB465, X...",5025.0,97.516010,5001,"IOUG579, OUAZ761, AUYM633, HLUL898, NCQW865, X...",2897,57.928414
2024-10-02,118010,40781.0,77229,Wed,8429.285714,5516.357143,2912.928571,30329,7692.0,22637,9255.0,0.0,9255.0,11376.0,0.0,11376.0,4855.0,0.0,4855.0,5645.0,0.0,5645.0,0.0,0.0,0.0,5582.0,0.0,5582.0,44.942801,31.427845,0.203373,14.502161,1.504110,0.000000,0.384713,0.190662,14.514112,14.495850,0.219632,0.219632,0.219632,0.050684,0.084474,30.0,0.0,139173,0.0,12033.0,714.0,377.0,0.0,132.0,0.0,0.0,4576.0,"BBBU890, BRKI251, UTJY519, VUNL729, FPLB522",4571.0,99.890734,5077.0,"KPEI715, CCUF236, ISSX669, MJGD070, QYHB461, I...",5007.0,98.621233,6312,"FHUA821, VQPT342, GREP070, PRUF680, FCGO241, Q...",5943,94.153992


In [316]:
res_final=res_final.reset_index()

In [334]:
res_final=res_final.rename(columns={'call_date':'date_day'})

In [336]:
res_final.head(2)

,date_day,n_calls,n_queue_calls,n_ivr_calls,day_of_week,avg_calls_per_hour,avg_ivr_calls_per_hour,avg_queue_calls_per_hour,n_calls_ln0,n_calls_ln0_queue,n_calls_ln0_ivr,n_calls_legal,n_ivr_calls_legal,n_queue_calls_legal,n_calls_mid_priority,n_ivr_calls_mid_priority,n_queue_calls_mid_priority,n_calls_initial,n_ivr_calls_initial,n_queue_calls_initial,n_calls_low_priority,n_ivr_calls_low_priority,n_queue_calls_low_priority,n_calls_high_priority,n_ivr_calls_high_priority,n_queue_calls_high_priority,n_calls_mixed_priority,n_ivr_calls_mixed_priority,n_queue_calls_mixed_priority,perc_calls_busy,perc_calls_no_answer,perc_calls_congestion,perc_calls_answer,perc_calls_abandoned,perc_calls_global,perc_calls_no_such_number,perc_calls_drop,perc_calls_answer_queue,perc_calls_answer_ivr,perc_calls_with_disposition1,perc_calls_with_disposition2,perc_calls_with_disposition3,perc_calls_with_disposition1_reschedule,perc_calls_with_disposition2_PTP,n_manual_outbound_calls,n_inbound_calls,n_sms_sent,n_sms_sent_dpd_m3,n_sms_sent_dpd_m1,n_sms_sent_dpd_0,n_sms_sent_dpd_1,n_sms_sent_dpd_3,n_sms_sent_dpd_7,n_sms_sent_dpd_18,n_sms_sent_dpd_25,n_loans_expected_calls_dpd3_15,loans_expected_not_called_dpd3_15,n_loans_called_dpd3_15,perc_loans_called_dpd3_15,n_loans_expected_calls_dpd16_30,loans_expected_not_called_dpd16_30,n_loans_called_dpd16_30,perc_loans_called_dpd16_30,n_loans_expected_calls_dpd0_2,loans_expected_not_called_dpd0_2,n_loans_called_dpd0_2,perc_loans_called_dpd0_2
0,2024-10-01,108847,42129.0,66718,Tue,7774.79,4765.57,3009.21,25607,8065.0,17542,10248.0,0.0,10248.0,11820.0,0.0,11820.0,5265.0,0.0,5265.0,6254.0,0.0,6254.0,0.0,0.0,0.0,3759.0,0.0,3759.0,48.38,29.10,0.19,14.52,1.7,0.0,0.30,0.18,12.71,15.66,0.07,0.07,0.07,0.04,0.04,49.0,0.0,141521,4129.0,11205.0,457.0,800.0,0.0,288.0,126.0,90.0,5201.0,"QSZC433, LGXC182, WEYL321, SFAC000, ZSFW368, F...",5113.0,98.31,5153.0,"UEEF073, DXAU816, VHVE316, LASB551, DKEB465, X...",5025.0,97.52,5001,"IOUG579, OUAZ761, AUYM633, HLUL898, NCQW865, X...",2897,57.93
1,2024-10-02,118010,40781.0,77229,Wed,8429.29,5516.36,2912.93,30329,7692.0,22637,9255.0,0.0,9255.0,11376.0,0.0,11376.0,4855.0,0.0,4855.0,5645.0,0.0,5645.0,0.0,0.0,0.0,5582.0,0.0,5582.0,44.94,31.43,0.20,14.50,1.5,0.0,0.38,0.19,14.51,14.50,0.22,0.22,0.22,0.05,0.08,30.0,0.0,139173,0.0,12033.0,714.0,377.0,0.0,132.0,0.0,0.0,4576.0,"BBBU890, BRKI251, UTJY519, VUNL729, FPLB522",4571.0,99.89,5077.0,"KPEI715, CCUF236, ISSX669, MJGD070, QYHB461, I...",5007.0,98.62,6312,"FHUA821, VQPT342, GREP070, PRUF680, FCGO241, Q...",5943,94.15


In [322]:
my_user="AMIT_S"
my_pass="Amit2525!!"
my_account="xukvgcv-xm56910"
my_warehouse='ANALYTICS_WH'
my_db='ghana_prod'
table_name='collections_blood_tests_daily'

In [324]:
import snowflake.connector
from snowflake.sqlalchemy import URL
from sqlalchemy import create_engine

In [326]:
engine = create_engine(URL(user=my_user,password=my_pass,account=my_account,warehouse=my_warehouse,database=my_db))
connection = engine.connect()

In [330]:
res_final=res_final.round(2)

In [338]:
# # # FIRST UPLOAD ONLY
# ress.to_sql(table_name, connection, schema='risk_reports',if_exists='replace', index=False)
res_final.to_sql(table_name, connection, schema='risk_reports',if_exists='append', index=False)


61

# Weekly

In [340]:
df=xc_data

In [342]:
# # Calculate number of calls per team per day
calls_per_team2 = df.groupby(['call_weekday','call_type', 'campaign_short']).size().reset_index(name='n_calls_per_campaign').fillna(0)
calls_per_team2 = pd.pivot_table(calls_per_team2, values='n_calls_per_campaign', index=['call_weekday', 'campaign_short'], columns=['call_type'], fill_value=0).reset_index()
calls_per_team2['total'] = calls_per_team2[['ivr', 'queue']].sum(axis=1)
calls_per_team2.columns.name = None

calls_per_team2.tail(4)

,call_weekday,campaign_short,ivr,queue,total
41,2024-11-18,mixed_priority,0.0,3039.0,3039.0
42,2024-11-18,other,348027.0,122408.0,470435.0
43,2024-11-25,legal,0.0,63550.0,63550.0
44,2024-11-25,other,416224.0,165295.0,581519.0


In [343]:
calls_per_hour_per_week = df.groupby(['call_weekday', 'call_hour']).size().reset_index(name='n_calls_per_hour')
ivr_calls_per_hour_per_week = df[df.call_type=='ivr'].groupby(['call_weekday', 'call_hour']).size().reset_index(name='n_ivr_calls_per_hour')
queue_calls_per_hour_per_week = df[df.call_type=='queue'].groupby(['call_weekday', 'call_hour']).size().reset_index(name='n_queue_calls_per_hour')

In [344]:
# Create Res
res2=pd.DataFrame(df.groupby('call_weekday').id.nunique()).rename(columns={'id':'n_calls'})
res2['n_queue_calls']=df[df.call_type=='queue'].groupby('call_weekday').id.nunique()
res2['n_ivr_calls']=df[df.call_type=='ivr'].groupby('call_weekday').id.nunique()
res2 = res2.reset_index()
res2=res2.set_index('call_weekday')
res2['avg_calls_per_hour']=calls_per_hour_per_week.groupby('call_weekday').n_calls_per_hour.mean()
res2['avg_ivr_calls_per_hour']=ivr_calls_per_hour_per_week.groupby('call_weekday').n_ivr_calls_per_hour.mean()
res2['avg_queue_calls_per_hour']=queue_calls_per_hour_per_week.groupby('call_weekday').n_queue_calls_per_hour.mean()
res2.head(2)

,n_calls,n_queue_calls,n_ivr_calls,avg_calls_per_hour,avg_ivr_calls_per_hour,avg_queue_calls_per_hour
call_weekday,,,,,,
2024-09-30,571572,205750,365822,40826.571429,26130.142857,14696.428571
2024-10-07,437429,144880,292549,31244.928571,20896.357143,10348.571429


In [345]:
calls_per_team2=calls_per_team2.set_index('call_weekday')

In [346]:
# Add Calls Per Team
teams = ['legal', 'mid_priority', 'initial', 'low_priority','high_priority','mixed_priority']
for team in teams:
    res2 = process_team_data(team, res2, calls_per_team2)

In [347]:
res2.head(2)

,n_calls,n_queue_calls,n_ivr_calls,avg_calls_per_hour,avg_ivr_calls_per_hour,avg_queue_calls_per_hour,n_calls_legal,n_ivr_calls_legal,n_queue_calls_legal,n_calls_mid_priority,n_ivr_calls_mid_priority,n_queue_calls_mid_priority,n_calls_initial,n_ivr_calls_initial,n_queue_calls_initial,n_calls_low_priority,n_ivr_calls_low_priority,n_queue_calls_low_priority,n_calls_high_priority,n_ivr_calls_high_priority,n_queue_calls_high_priority,n_calls_mixed_priority,n_ivr_calls_mixed_priority,n_queue_calls_mixed_priority
call_weekday,,,,,,,,,,,,,,,,,,,,,,,,
2024-09-30,571572,205750,365822,40826.571429,26130.142857,14696.428571,48666.0,0.0,48666.0,55692.0,0.0,55692.0,24707.0,0.0,24707.0,27117.0,0.0,27117.0,NaN,NaN,NaN,29331.0,0.0,29331.0
2024-10-07,437429,144880,292549,31244.928571,20896.357143,10348.571429,32927.0,0.0,32927.0,13948.0,0.0,13948.0,6314.0,0.0,6314.0,7113.0,0.0,7113.0,NaN,NaN,NaN,11385.0,0.0,11385.0


In [348]:
# state desc
res2['perc_calls_busy']=df[df.state_desc=='Busy'].groupby('call_weekday').id.nunique()/res2.n_calls*100
res2['perc_calls_no_answer']=df[df.state_desc=='NoAnswer'].groupby('call_weekday').id.nunique()/res2.n_calls*100
res2['perc_calls_congestion']=df[df.state_desc=='Congestion'].groupby('call_weekday').id.nunique()/res2.n_calls*100
res2['perc_calls_answer']=df[df.state_desc=='Answer'].groupby('call_weekday').id.nunique()/res2.n_calls*100
res2['perc_calls_abandoned']=df[df.state_desc=='Abandoned'].groupby('call_weekday').id.nunique()/res2.n_calls*100
res2['perc_calls_global']=df[df.state_desc=='Global'].groupby('call_weekday').id.nunique()/res2.n_calls*100
res2['perc_calls_no_such_number']=df[df.state_desc=='NoSuchNumber'].groupby('call_weekday').id.nunique()/res2.n_calls*100
res2['perc_calls_drop']=df[df.state_desc=='Drop'].groupby('call_weekday').id.nunique()/res2.n_calls*100

In [349]:
res2['n_calls_answer_queue']=df[(df.state_desc=='Answer') & (df.call_type=='queue')].groupby('call_weekday').id.nunique()
res2['n_calls_answer_ivr']=df[(df.state_desc=='Answer') & (df.call_type=='ivr')].groupby('call_weekday').id.nunique()


# dispositions
res2['n_calls_with_disposition1']=df[(df.state_desc=='Answer')&(df.disposition.notna())].groupby('call_weekday').id.nunique()
res2['n_calls_with_disposition2']=df[(df.state_desc=='Answer')&(df.disposition_2.notna())].groupby('call_weekday').id.nunique()
res2['n_calls_with_disposition3']=df[(df.state_desc=='Answer')&(df.disposition_3.notna())].groupby('call_weekday').id.nunique()

res2['perc_calls_with_disposition1']=res2.n_calls_with_disposition1/res2.n_calls_answer_queue*100
res2['perc_calls_with_disposition2']=res2.n_calls_with_disposition2/res2.n_calls_answer_queue*100
res2['perc_calls_with_disposition3']=res2.n_calls_with_disposition3/res2.n_calls_answer_queue*100

res2['perc_calls_with_disposition1_reschedule']=df[(df.state_desc=='Answer')&((df.disposition=='reschedule')|(df.disposition=='recallme'))].groupby('call_weekday').id.nunique()/res2.n_calls_answer_queue*100
res2['perc_calls_with_disposition2_PTP']=df[(df.state_desc=='Answer')&(df.disposition_2=='Promise to pay')].groupby('call_weekday').id.nunique()/res2.n_calls_answer_queue*100



In [350]:
res2=res2.drop(columns=['n_calls_with_disposition1','n_calls_with_disposition2','n_calls_with_disposition3','n_calls_answer_queue','n_calls_answer_ivr'])

In [351]:
res2=res2.fillna(0)

In [352]:
res2.head(2)

,n_calls,n_queue_calls,n_ivr_calls,avg_calls_per_hour,avg_ivr_calls_per_hour,avg_queue_calls_per_hour,n_calls_legal,n_ivr_calls_legal,n_queue_calls_legal,n_calls_mid_priority,n_ivr_calls_mid_priority,n_queue_calls_mid_priority,n_calls_initial,n_ivr_calls_initial,n_queue_calls_initial,n_calls_low_priority,n_ivr_calls_low_priority,n_queue_calls_low_priority,n_calls_high_priority,n_ivr_calls_high_priority,n_queue_calls_high_priority,n_calls_mixed_priority,n_ivr_calls_mixed_priority,n_queue_calls_mixed_priority,perc_calls_busy,perc_calls_no_answer,perc_calls_congestion,perc_calls_answer,perc_calls_abandoned,perc_calls_global,perc_calls_no_such_number,perc_calls_drop,perc_calls_with_disposition1,perc_calls_with_disposition2,perc_calls_with_disposition3,perc_calls_with_disposition1_reschedule,perc_calls_with_disposition2_PTP
call_weekday,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2024-09-30,571572,205750,365822,40826.571429,26130.142857,14696.428571,48666.0,0.0,48666.0,55692.0,0.0,55692.0,24707.0,0.0,24707.0,27117.0,0.0,27117.0,0.0,0.0,0.0,29331.0,0.0,29331.0,49.305599,29.954406,0.266808,12.891464,1.521768,0.001925,0.288678,0.180730,0.698513,0.698513,0.698513,0.063843,0.165240
2024-10-07,437429,144880,292549,31244.928571,20896.357143,10348.571429,32927.0,0.0,32927.0,13948.0,0.0,13948.0,6314.0,0.0,6314.0,7113.0,0.0,7113.0,0.0,0.0,0.0,11385.0,0.0,11385.0,49.362982,32.346964,0.185173,10.804496,1.025081,0.001600,0.694284,0.147681,0.656678,0.650303,0.650303,0.070131,0.318776


In [353]:
res2.index=res2.index.astype('str')

In [354]:
res2.shape

(9, 37)

In [355]:
res2=res2.join(add22)

In [356]:
res2=res2.join(add33)

In [357]:
res2.n_manual_outbound_calls=res2.n_manual_outbound_calls.fillna(0)
res2.n_inbound_calls=res2.n_inbound_calls.fillna(0)
res2.n_sms_sent=res2.n_sms_sent.fillna(0)

In [358]:
lb_hist.period=lb_hist.period.astype('str')

In [359]:
# add week to it and sum weekly expected, caleld, not called
add_weekly1=add5_1.set_index('period').join(lb_hist[['period','call_week']].set_index('period'))
add_weekly1=add_weekly1.reset_index()[['call_week','n_loans_expected_calls_dpd3_15','n_loans_called_dpd3_15']].drop_duplicates()

add_weekly2=add5_2.set_index('period').join(lb_hist[['period','call_week']].set_index('period'))
add_weekly2=add_weekly2.reset_index()[['call_week','n_loans_expected_calls_dpd16_30','n_loans_called_dpd16_30']].drop_duplicates()

add_weekly3=add5_3.set_index('period').join(lb_hist[['period','call_week']].set_index('period'))
add_weekly3=add_weekly3.reset_index()[['call_week','n_loans_expected_calls_dpd0_2','n_loans_called_dpd0_2']].drop_duplicates()


In [360]:
res2['n_loans_expected_calls_dpd0_2']=add_weekly3.groupby('call_week').n_loans_expected_calls_dpd0_2.sum()
res2['n_loans_called_dpd0_2']=add_weekly3.groupby('call_week').n_loans_called_dpd0_2.sum()
res2['perc_loans_called_dpd0_2']=res2.n_loans_called_dpd0_2/res2.n_loans_expected_calls_dpd0_2*100

res2['n_loans_expected_calls_dpd3_15']=add_weekly1.groupby('call_week').n_loans_expected_calls_dpd3_15.sum()
res2['n_loans_called_dpd3_15']=add_weekly1.groupby('call_week').n_loans_called_dpd3_15.sum()
res2['perc_loans_called_dpd3_15']=res2.n_loans_called_dpd3_15/res2.n_loans_expected_calls_dpd3_15*100

res2['n_loans_expected_calls_dpd16_30']=add_weekly2.groupby('call_week').n_loans_expected_calls_dpd16_30.sum()
res2['n_loans_called_dpd16_30']=add_weekly2.groupby('call_week').n_loans_called_dpd16_30.sum()
res2['perc_loans_called_dpd16_30']=res2.n_loans_called_dpd16_30/res2.n_loans_expected_calls_dpd16_30*100



In [361]:
res2

,n_calls,n_queue_calls,n_ivr_calls,avg_calls_per_hour,avg_ivr_calls_per_hour,avg_queue_calls_per_hour,n_calls_legal,n_ivr_calls_legal,n_queue_calls_legal,n_calls_mid_priority,n_ivr_calls_mid_priority,n_queue_calls_mid_priority,n_calls_initial,n_ivr_calls_initial,n_queue_calls_initial,n_calls_low_priority,n_ivr_calls_low_priority,n_queue_calls_low_priority,n_calls_high_priority,n_ivr_calls_high_priority,n_queue_calls_high_priority,n_calls_mixed_priority,n_ivr_calls_mixed_priority,n_queue_calls_mixed_priority,perc_calls_busy,perc_calls_no_answer,perc_calls_congestion,perc_calls_answer,perc_calls_abandoned,perc_calls_global,perc_calls_no_such_number,perc_calls_drop,perc_calls_with_disposition1,perc_calls_with_disposition2,perc_calls_with_disposition3,perc_calls_with_disposition1_reschedule,perc_calls_with_disposition2_PTP,n_manual_outbound_calls,n_inbound_calls,n_sms_sent,n_sms_sent_dpd_m3,n_sms_sent_dpd_m1,n_sms_sent_dpd_0,n_sms_sent_dpd_1,n_sms_sent_dpd_3,n_sms_sent_dpd_7,n_sms_sent_dpd_18,n_sms_sent_dpd_25,n_loans_expected_calls_dpd0_2,n_loans_called_dpd0_2,perc_loans_called_dpd0_2,n_loans_expected_calls_dpd3_15,n_loans_called_dpd3_15,perc_loans_called_dpd3_15,n_loans_expected_calls_dpd16_30,n_loans_called_dpd16_30,perc_loans_called_dpd16_30
call_weekday,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2024-09-30,571572,205750,365822,40826.571429,26130.142857,14696.428571,48666.0,0.0,48666.0,55692.0,0.0,55692.0,24707.0,0.0,24707.0,27117.0,0.0,27117.0,0.0,0.0,0.0,29331.0,0.0,29331.0,49.305599,29.954406,0.266808,12.891464,1.521768,0.001925,0.288678,0.180730,0.698513,0.698513,0.698513,0.063843,0.165240,233,0.0,655369,7234,33003,3430.0,3759,6217,812.0,267,221,29070,24170,83.144135,30070,29674,98.683073,31602,30364,96.082526
2024-10-07,437429,144880,292549,31244.928571,20896.357143,10348.571429,32927.0,0.0,32927.0,13948.0,0.0,13948.0,6314.0,0.0,6314.0,7113.0,0.0,7113.0,0.0,0.0,0.0,11385.0,0.0,11385.0,49.362982,32.346964,0.185173,10.804496,1.025081,0.001600,0.694284,0.147681,0.656678,0.650303,0.650303,0.070131,0.318776,98,0.0,464516,1853,6952,1116.0,785,2296,1339.0,95,296,10817,9069,83.840251,43820,41668,95.089000,27410,24330,88.763225
2024-10-14,395631,99135,296496,28259.357143,21178.285714,7081.071429,22235.0,0.0,22235.0,7045.0,0.0,7045.0,2873.0,0.0,2873.0,3171.0,0.0,3171.0,0.0,0.0,0.0,5700.0,0.0,5700.0,48.093046,33.219844,0.206000,9.394107,0.723401,0.003033,1.289080,0.125369,1.327337,1.305397,1.305397,0.219394,0.482668,33,0.0,467167,1994,6444,953.0,660,2327,543.0,605,41,9256,9094,98.249784,29718,28940,97.382058,44458,39387,88.593729
2024-10-21,454501,157290,297211,32464.357143,21229.357143,11235.000000,40617.0,0.0,40617.0,41092.0,0.0,41092.0,15851.0,0.0,15851.0,14420.0,0.0,14420.0,0.0,0.0,0.0,31514.0,0.0,31514.0,46.797477,33.658672,0.175577,10.219779,1.072165,0.000440,0.396479,0.157975,0.328450,0.328450,0.328450,0.059121,0.118242,101,0.0,394768,10866,11347,640.0,329,1910,173.0,223,172,9948,9594,96.441496,19176,19108,99.645390,48627,48057,98.827812
2024-10-28,560744,218931,341813,40053.142857,24415.214286,15637.928571,55306.0,0.0,55306.0,66319.0,0.0,66319.0,4265.0,0.0,4265.0,30692.0,0.0,30692.0,0.0,0.0,0.0,31574.0,0.0,31574.0,45.406460,31.545054,0.273743,12.532635,1.371392,0.011413,0.407673,0.201161,1.232673,1.223642,1.223642,0.139974,0.298009,735,0.0,1041999,23159,39044,5354.0,4394,8236,1263.0,795,1041,31431,26715,84.995705,29569,29246,98.907640,30987,30403,98.115339
2024-11-04,666536,217909,448627,47609.714286,32044.785714,15564.928571,59260.0,0.0,59260.0,32271.0,0.0,32271.0,0.0,0.0,0.0,13511.0,0.0,13511.0,0.0,0.0,0.0,23708.0,0.0,23708.0,36.374779,22.822623,0.148829,8.440954,1.103766,0.005251,26.880769,0.115823,0.885924,0.869518,0.869518,0.158591,0.371869,192,0.0,703379,3492,15356,4278.0,2878,6958,2867.0,727,736,21283,19509,91.664709,43588,37136,85.197761,29325,24808,84.596760
2024-11-11,553383,189752,363631,39527.357143,25973.642857,14596.307692,58375.0,0.0,58375.0,6800.0,0.0,6800.0,0.0,0.0,0.0,2636.0,0.0,2636.0,0.0,0.0

In [362]:
res2.tail(6)

,n_calls,n_queue_calls,n_ivr_calls,avg_calls_per_hour,avg_ivr_calls_per_hour,avg_queue_calls_per_hour,n_calls_legal,n_ivr_calls_legal,n_queue_calls_legal,n_calls_mid_priority,n_ivr_calls_mid_priority,n_queue_calls_mid_priority,n_calls_initial,n_ivr_calls_initial,n_queue_calls_initial,n_calls_low_priority,n_ivr_calls_low_priority,n_queue_calls_low_priority,n_calls_high_priority,n_ivr_calls_high_priority,n_queue_calls_high_priority,n_calls_mixed_priority,n_ivr_calls_mixed_priority,n_queue_calls_mixed_priority,perc_calls_busy,perc_calls_no_answer,perc_calls_congestion,perc_calls_answer,perc_calls_abandoned,perc_calls_global,perc_calls_no_such_number,perc_calls_drop,perc_calls_with_disposition1,perc_calls_with_disposition2,perc_calls_with_disposition3,perc_calls_with_disposition1_reschedule,perc_calls_with_disposition2_PTP,n_manual_outbound_calls,n_inbound_calls,n_sms_sent,n_sms_sent_dpd_m3,n_sms_sent_dpd_m1,n_sms_sent_dpd_0,n_sms_sent_dpd_1,n_sms_sent_dpd_3,n_sms_sent_dpd_7,n_sms_sent_dpd_18,n_sms_sent_dpd_25,n_loans_expected_calls_dpd0_2,n_loans_called_dpd0_2,perc_loans_called_dpd0_2,n_loans_expected_calls_dpd3_15,n_loans_called_dpd3_15,perc_loans_called_dpd3_15,n_loans_expected_calls_dpd16_30,n_loans_called_dpd16_30,perc_loans_called_dpd16_30
call_weekday,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2024-10-21,454501,157290,297211,32464.357143,21229.357143,11235.000000,40617.0,0.0,40617.0,41092.0,0.0,41092.0,15851.0,0.0,15851.0,14420.0,0.0,14420.0,0.0,0.0,0.0,31514.0,0.0,31514.0,46.797477,33.658672,0.175577,10.219779,1.072165,0.000440,0.396479,0.157975,0.328450,0.328450,0.328450,0.059121,0.118242,101,0.0,394768,10866,11347,640.0,329,1910,173.0,223,172,9948,9594,96.441496,19176,19108,99.645390,48627,48057,98.827812
2024-10-28,560744,218931,341813,40053.142857,24415.214286,15637.928571,55306.0,0.0,55306.0,66319.0,0.0,66319.0,4265.0,0.0,4265.0,30692.0,0.0,30692.0,0.0,0.0,0.0,31574.0,0.0,31574.0,45.406460,31.545054,0.273743,12.532635,1.371392,0.011413,0.407673,0.201161,1.232673,1.223642,1.223642,0.139974,0.298009,735,0.0,1041999,23159,39044,5354.0,4394,8236,1263.0,795,1041,31431,26715,84.995705,29569,29246,98.907640,30987,30403,98.115339
2024-11-04,666536,217909,448627,47609.714286,32044.785714,15564.928571,59260.0,0.0,59260.0,32271.0,0.0,32271.0,0.0,0.0,0.0,13511.0,0.0,13511.0,0.0,0.0,0.0,23708.0,0.0,23708.0,36.374779,22.822623,0.148829,8.440954,1.103766,0.005251,26.880769,0.115823,0.885924,0.869518,0.869518,0.158591,0.371869,192,0.0,703379,3492,15356,4278.0,2878,6958,2867.0,727,736,21283,19509,91.664709,43588,37136,85.197761,29325,24808,84.596760
2024-11-11,553383,189752,363631,39527.357143,25973.642857,14596.307692,58375.0,0.0,58375.0,6800.0,0.0,6800.0,0.0,0.0,0.0,2636.0,0.0,2636.0,0.0,0.0,0.0,36824.0,0.0,36824.0,45.980090,24.834879,0.113845,8.131077,1.243985,0.000181,15.706120,0.138241,0.000000,0.000000,0.000000,0.000000,0.000000,175,0.0,308598,3353,9855,863.0,1715,2806,1021.0,289,199,10657,10497,98.498639,41035,37466,91.302547,35028,28910,82.533973
2024-11-18,520934,172907,348027,37209.571429,24859.071429,13300.538462,47460.0,0.0,47460.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3039.0,0.0,3039.0,44.441330,32.760196,0.088303,12.618297,1.784295,0.036473,0.870936,0.200601,7.842195,7.751441,7.751441,0.976938,2.829383,97,0.0,720961,6202,10488,2247.0,3717,2681,1166.0,1981,497,10093,9872,97.810364,23664,23616,99.797160,50196,50060,99.729062
2024-11-25,645069,228845,416224,46076.357143,29730.285714,17603.461538,63550.0,0.0,63550.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,46.024379,30.024230,0.146186,14.961500,2.088459,0.003255,0.547848,0.292062,9.161390,9.019321,9.019321,1.359799,3.774963,532,0.0,974043,14989,27884,4648.0,2869,4155,1007.0,488,1354,20889,19808,94.825028,21482,21404,99.636905,39412,39019,99.002842


### Upload to SF

In [386]:
res_final=res2

In [388]:
res_final.head(2)

,n_calls,n_queue_calls,n_ivr_calls,avg_calls_per_hour,avg_ivr_calls_per_hour,avg_queue_calls_per_hour,n_calls_legal,n_ivr_calls_legal,n_queue_calls_legal,n_calls_mid_priority,n_ivr_calls_mid_priority,n_queue_calls_mid_priority,n_calls_initial,n_ivr_calls_initial,n_queue_calls_initial,n_calls_low_priority,n_ivr_calls_low_priority,n_queue_calls_low_priority,n_calls_high_priority,n_ivr_calls_high_priority,n_queue_calls_high_priority,n_calls_mixed_priority,n_ivr_calls_mixed_priority,n_queue_calls_mixed_priority,perc_calls_busy,perc_calls_no_answer,perc_calls_congestion,perc_calls_answer,perc_calls_abandoned,perc_calls_global,perc_calls_no_such_number,perc_calls_drop,perc_calls_with_disposition1,perc_calls_with_disposition2,perc_calls_with_disposition3,perc_calls_with_disposition1_reschedule,perc_calls_with_disposition2_PTP,n_manual_outbound_calls,n_inbound_calls,n_sms_sent,n_sms_sent_dpd_m3,n_sms_sent_dpd_m1,n_sms_sent_dpd_0,n_sms_sent_dpd_1,n_sms_sent_dpd_3,n_sms_sent_dpd_7,n_sms_sent_dpd_18,n_sms_sent_dpd_25,n_loans_expected_calls_dpd0_2,n_loans_called_dpd0_2,perc_loans_called_dpd0_2,n_loans_expected_calls_dpd3_15,n_loans_called_dpd3_15,perc_loans_called_dpd3_15,n_loans_expected_calls_dpd16_30,n_loans_called_dpd16_30,perc_loans_called_dpd16_30
call_weekday,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2024-09-30,571572,205750,365822,40826.571429,26130.142857,14696.428571,48666.0,0.0,48666.0,55692.0,0.0,55692.0,24707.0,0.0,24707.0,27117.0,0.0,27117.0,0.0,0.0,0.0,29331.0,0.0,29331.0,49.305599,29.954406,0.266808,12.891464,1.521768,0.001925,0.288678,0.180730,0.698513,0.698513,0.698513,0.063843,0.165240,233,0.0,655369,7234,33003,3430.0,3759,6217,812.0,267,221,29070,24170,83.144135,30070,29674,98.683073,31602,30364,96.082526
2024-10-07,437429,144880,292549,31244.928571,20896.357143,10348.571429,32927.0,0.0,32927.0,13948.0,0.0,13948.0,6314.0,0.0,6314.0,7113.0,0.0,7113.0,0.0,0.0,0.0,11385.0,0.0,11385.0,49.362982,32.346964,0.185173,10.804496,1.025081,0.001600,0.694284,0.147681,0.656678,0.650303,0.650303,0.070131,0.318776,98,0.0,464516,1853,6952,1116.0,785,2296,1339.0,95,296,10817,9069,83.840251,43820,41668,95.089000,27410,24330,88.763225


In [390]:
res_final.head(2)

,n_calls,n_queue_calls,n_ivr_calls,avg_calls_per_hour,avg_ivr_calls_per_hour,avg_queue_calls_per_hour,n_calls_legal,n_ivr_calls_legal,n_queue_calls_legal,n_calls_mid_priority,n_ivr_calls_mid_priority,n_queue_calls_mid_priority,n_calls_initial,n_ivr_calls_initial,n_queue_calls_initial,n_calls_low_priority,n_ivr_calls_low_priority,n_queue_calls_low_priority,n_calls_high_priority,n_ivr_calls_high_priority,n_queue_calls_high_priority,n_calls_mixed_priority,n_ivr_calls_mixed_priority,n_queue_calls_mixed_priority,perc_calls_busy,perc_calls_no_answer,perc_calls_congestion,perc_calls_answer,perc_calls_abandoned,perc_calls_global,perc_calls_no_such_number,perc_calls_drop,perc_calls_with_disposition1,perc_calls_with_disposition2,perc_calls_with_disposition3,perc_calls_with_disposition1_reschedule,perc_calls_with_disposition2_PTP,n_manual_outbound_calls,n_inbound_calls,n_sms_sent,n_sms_sent_dpd_m3,n_sms_sent_dpd_m1,n_sms_sent_dpd_0,n_sms_sent_dpd_1,n_sms_sent_dpd_3,n_sms_sent_dpd_7,n_sms_sent_dpd_18,n_sms_sent_dpd_25,n_loans_expected_calls_dpd0_2,n_loans_called_dpd0_2,perc_loans_called_dpd0_2,n_loans_expected_calls_dpd3_15,n_loans_called_dpd3_15,perc_loans_called_dpd3_15,n_loans_expected_calls_dpd16_30,n_loans_called_dpd16_30,perc_loans_called_dpd16_30
call_weekday,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2024-09-30,571572,205750,365822,40826.571429,26130.142857,14696.428571,48666.0,0.0,48666.0,55692.0,0.0,55692.0,24707.0,0.0,24707.0,27117.0,0.0,27117.0,0.0,0.0,0.0,29331.0,0.0,29331.0,49.305599,29.954406,0.266808,12.891464,1.521768,0.001925,0.288678,0.180730,0.698513,0.698513,0.698513,0.063843,0.165240,233,0.0,655369,7234,33003,3430.0,3759,6217,812.0,267,221,29070,24170,83.144135,30070,29674,98.683073,31602,30364,96.082526
2024-10-07,437429,144880,292549,31244.928571,20896.357143,10348.571429,32927.0,0.0,32927.0,13948.0,0.0,13948.0,6314.0,0.0,6314.0,7113.0,0.0,7113.0,0.0,0.0,0.0,11385.0,0.0,11385.0,49.362982,32.346964,0.185173,10.804496,1.025081,0.001600,0.694284,0.147681,0.656678,0.650303,0.650303,0.070131,0.318776,98,0.0,464516,1853,6952,1116.0,785,2296,1339.0,95,296,10817,9069,83.840251,43820,41668,95.089000,27410,24330,88.763225


In [392]:
res_final=res_final.reset_index()

In [394]:
res_final=res_final.rename(columns={'call_weekday':'date_week'})

In [396]:
res_final.head(2)

,date_week,n_calls,n_queue_calls,n_ivr_calls,avg_calls_per_hour,avg_ivr_calls_per_hour,avg_queue_calls_per_hour,n_calls_legal,n_ivr_calls_legal,n_queue_calls_legal,n_calls_mid_priority,n_ivr_calls_mid_priority,n_queue_calls_mid_priority,n_calls_initial,n_ivr_calls_initial,n_queue_calls_initial,n_calls_low_priority,n_ivr_calls_low_priority,n_queue_calls_low_priority,n_calls_high_priority,n_ivr_calls_high_priority,n_queue_calls_high_priority,n_calls_mixed_priority,n_ivr_calls_mixed_priority,n_queue_calls_mixed_priority,perc_calls_busy,perc_calls_no_answer,perc_calls_congestion,perc_calls_answer,perc_calls_abandoned,perc_calls_global,perc_calls_no_such_number,perc_calls_drop,perc_calls_with_disposition1,perc_calls_with_disposition2,perc_calls_with_disposition3,perc_calls_with_disposition1_reschedule,perc_calls_with_disposition2_PTP,n_manual_outbound_calls,n_inbound_calls,n_sms_sent,n_sms_sent_dpd_m3,n_sms_sent_dpd_m1,n_sms_sent_dpd_0,n_sms_sent_dpd_1,n_sms_sent_dpd_3,n_sms_sent_dpd_7,n_sms_sent_dpd_18,n_sms_sent_dpd_25,n_loans_expected_calls_dpd0_2,n_loans_called_dpd0_2,perc_loans_called_dpd0_2,n_loans_expected_calls_dpd3_15,n_loans_called_dpd3_15,perc_loans_called_dpd3_15,n_loans_expected_calls_dpd16_30,n_loans_called_dpd16_30,perc_loans_called_dpd16_30
0,2024-09-30,571572,205750,365822,40826.571429,26130.142857,14696.428571,48666.0,0.0,48666.0,55692.0,0.0,55692.0,24707.0,0.0,24707.0,27117.0,0.0,27117.0,0.0,0.0,0.0,29331.0,0.0,29331.0,49.305599,29.954406,0.266808,12.891464,1.521768,0.001925,0.288678,0.180730,0.698513,0.698513,0.698513,0.063843,0.165240,233,0.0,655369,7234,33003,3430.0,3759,6217,812.0,267,221,29070,24170,83.144135,30070,29674,98.683073,31602,30364,96.082526
1,2024-10-07,437429,144880,292549,31244.928571,20896.357143,10348.571429,32927.0,0.0,32927.0,13948.0,0.0,13948.0,6314.0,0.0,6314.0,7113.0,0.0,7113.0,0.0,0.0,0.0,11385.0,0.0,11385.0,49.362982,32.346964,0.185173,10.804496,1.025081,0.001600,0.694284,0.147681,0.656678,0.650303,0.650303,0.070131,0.318776,98,0.0,464516,1853,6952,1116.0,785,2296,1339.0,95,296,10817,9069,83.840251,43820,41668,95.089000,27410,24330,88.763225


In [398]:
my_user="AMIT_S"
my_pass="Amit2525!!"
my_account="xukvgcv-xm56910"
my_warehouse='ANALYTICS_WH'
my_db='ghana_prod'
table_name='collections_blood_tests_weekly'

In [400]:
import snowflake.connector
from snowflake.sqlalchemy import URL
from sqlalchemy import create_engine

In [402]:
engine = create_engine(URL(user=my_user,password=my_pass,account=my_account,warehouse=my_warehouse,database=my_db))
connection = engine.connect()

In [403]:
res_final=res_final.round(2)

In [404]:
# # # FIRST UPLOAD ONLY
# ress.to_sql(table_name, connection, schema='risk_reports',if_exists='replace', index=False)
res_final.to_sql(table_name, connection, schema='risk_reports',if_exists='append', index=False)


9

# Monthly

In [408]:
df.head(3)

,loan_id,call_date,ln,id,member_name,caller_id_num,caller_id_name,scheduled_at,start_time,response_time,answer_time,state_desc,campaign_name,call_type,disposition,disposition_2,disposition_3,date,created_at,installment_identifier,phone,team,contact_id,data_origin,end_time,call_week,call_month,call_weekday,answered_ivr,answered_queue,answered_yesterday,answered_two_days,campaign_short,call_hour,period,days_late,is_exclusion_DPD_16_30,is_exclusion_DPD_3_15
0,054218766,2024-09-28,NaN,NaN,NaN,NaN,NaN,NaT,NaT,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaT,None,None,NaN,NaN,NaN,0.0,0.0,other,NaN,NaT,NaN,0,0
1,054218766,2024-09-29,NaN,NaN,NaN,NaN,NaN,NaT,NaT,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaT,None,None,NaN,NaN,NaN,0.0,0.0,other,NaN,NaT,NaN,0,0
2,054218766,2024-09-30,NaN,NaN,NaN,NaN,NaN,NaT,NaT,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaT,None,None,NaN,NaN,NaN,0.0,0.0,other,NaN,NaT,NaN,0,0


In [410]:
# # Calculate number of calls per team per day
calls_per_team3 = df.groupby(['call_month','call_type', 'campaign_short']).size().reset_index(name='n_calls_per_campaign').fillna(0)
calls_per_team3 = pd.pivot_table(calls_per_team3, values='n_calls_per_campaign', index=['call_month', 'campaign_short'], columns=['call_type'], fill_value=0).reset_index()
calls_per_team3['total'] = calls_per_team3[['ivr', 'queue']].sum(axis=1)
calls_per_team3.columns.name = None

calls_per_team3.tail(4)

,call_month,campaign_short,ivr,queue,total
13,2024-11-01,low_priority,0.0,22656.0,22656.0
14,2024-11-01,mid_priority,0.0,52565.0,52565.0
15,2024-11-01,mixed_priority,0.0,69931.0,69931.0
16,2024-11-01,other,2233805.0,506685.0,2740490.0


In [411]:
# Create Res
res3=pd.DataFrame(df.groupby('call_month').id.nunique()).rename(columns={'id':'n_calls'})
res3['n_queue_calls']=df[df.call_type=='queue'].groupby('call_month').id.nunique()
res3['n_ivr_calls']=df[df.call_type=='ivr'].groupby('call_month').id.nunique()
res3 = res3.reset_index()
res3=res3.set_index('call_month')
res3.head(2)

,n_calls,n_queue_calls,n_ivr_calls
call_month,,,
2024-09-01,165780,63682,102098
2024-10-01,2662638,741049,1921589


In [412]:
calls_per_team3=calls_per_team3.set_index('call_month')

In [413]:
# Add Calls Per Team
for team in teams:
    res3 = process_team_data(team, res3, calls_per_team3)

In [414]:
res3.head(2)

,n_calls,n_queue_calls,n_ivr_calls,n_calls_legal,n_ivr_calls_legal,n_queue_calls_legal,n_calls_mid_priority,n_ivr_calls_mid_priority,n_queue_calls_mid_priority,n_calls_initial,n_ivr_calls_initial,n_queue_calls_initial,n_calls_low_priority,n_ivr_calls_low_priority,n_queue_calls_low_priority,n_calls_high_priority,n_ivr_calls_high_priority,n_queue_calls_high_priority,n_calls_mixed_priority,n_ivr_calls_mixed_priority,n_queue_calls_mixed_priority
call_month,,,,,,,,,,,,,,,,,,,,,
2024-09-01,165780,63682,102098,17051.0,0.0,17051.0,8813.0,0.0,8813.0,4897.0,0.0,4897.0,4395.0,0.0,4395.0,NaN,NaN,NaN,7723.0,0.0,7723.0
2024-10-01,2662638,741049,1921589,178223.0,0.0,178223.0,161789.0,0.0,161789.0,49113.0,0.0,49113.0,71609.0,0.0,71609.0,NaN,NaN,NaN,95421.0,0.0,95421.0


In [415]:
# state desc
res3['perc_calls_busy']=df[df.state_desc=='Busy'].groupby('call_month').id.nunique()/res3.n_calls*100
res3['perc_calls_no_answer']=df[df.state_desc=='NoAnswer'].groupby('call_month').id.nunique()/res3.n_calls*100
res3['perc_calls_congestion']=df[df.state_desc=='Congestion'].groupby('call_month').id.nunique()/res3.n_calls*100
res3['perc_calls_answer']=df[df.state_desc=='Answer'].groupby('call_month').id.nunique()/res3.n_calls*100
res3['perc_calls_abandoned']=df[df.state_desc=='Abandoned'].groupby('call_month').id.nunique()/res3.n_calls*100
res3['perc_calls_global']=df[df.state_desc=='Global'].groupby('call_month').id.nunique()/res3.n_calls*100
res3['perc_calls_no_such_number']=df[df.state_desc=='NoSuchNumber'].groupby('call_month').id.nunique()/res3.n_calls*100
res3['perc_calls_drop']=df[df.state_desc=='Drop'].groupby('call_month').id.nunique()/res3.n_calls*100

In [416]:
res3['n_calls_answer_queue']=df[(df.state_desc=='Answer') & (df.call_type=='queue')].groupby('call_month').id.nunique()
res3['n_calls_answer_ivr']=df[(df.state_desc=='Answer') & (df.call_type=='ivr')].groupby('call_month').id.nunique()


# dispositions
res3['n_calls_with_disposition1']=df[(df.state_desc=='Answer')&(df.disposition.notna())].groupby('call_month').id.nunique()
res3['n_calls_with_disposition2']=df[(df.state_desc=='Answer')&(df.disposition_2.notna())].groupby('call_month').id.nunique()
res3['n_calls_with_disposition3']=df[(df.state_desc=='Answer')&(df.disposition_3.notna())].groupby('call_month').id.nunique()

res3['perc_calls_with_disposition1']=res3.n_calls_with_disposition1/res3.n_calls_answer_queue*100
res3['perc_calls_with_disposition2']=res3.n_calls_with_disposition2/res3.n_calls_answer_queue*100
res3['perc_calls_with_disposition3']=res3.n_calls_with_disposition3/res3.n_calls_answer_queue*100

res3['perc_calls_with_disposition1_reschedule']=df[(df.state_desc=='Answer')&((df.disposition=='reschedule')|(df.disposition=='recallme'))].groupby('call_month').id.nunique()/res3.n_calls_answer_queue*100
res3['perc_calls_with_disposition2_PTP']=df[(df.state_desc=='Answer')&(df.disposition_2=='Promise to pay')].groupby('call_month').id.nunique()/res3.n_calls_answer_queue*100



In [417]:
res3=res3.drop(columns=['n_calls_with_disposition1','n_calls_with_disposition2','n_calls_with_disposition3','n_calls_answer_queue','n_calls_answer_ivr'])

In [418]:
res3=res3.fillna(0)

In [419]:
res3=res3.join(add222)

In [420]:
res3=res3.join(add333)

In [421]:
res3

,n_calls,n_queue_calls,n_ivr_calls,n_calls_legal,n_ivr_calls_legal,n_queue_calls_legal,n_calls_mid_priority,n_ivr_calls_mid_priority,n_queue_calls_mid_priority,n_calls_initial,n_ivr_calls_initial,n_queue_calls_initial,n_calls_low_priority,n_ivr_calls_low_priority,n_queue_calls_low_priority,n_calls_high_priority,n_ivr_calls_high_priority,n_queue_calls_high_priority,n_calls_mixed_priority,n_ivr_calls_mixed_priority,n_queue_calls_mixed_priority,perc_calls_busy,perc_calls_no_answer,perc_calls_congestion,perc_calls_answer,perc_calls_abandoned,perc_calls_global,perc_calls_no_such_number,perc_calls_drop,perc_calls_with_disposition1,perc_calls_with_disposition2,perc_calls_with_disposition3,perc_calls_with_disposition1_reschedule,perc_calls_with_disposition2_PTP,n_manual_outbound_calls,n_inbound_calls,n_sms_sent,n_sms_sent_dpd_m3,n_sms_sent_dpd_m1,n_sms_sent_dpd_0,n_sms_sent_dpd_1,n_sms_sent_dpd_3,n_sms_sent_dpd_7,n_sms_sent_dpd_18,n_sms_sent_dpd_25
call_month,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2024-09-01,165780,63682,102098,17051.0,0.0,17051.0,8813.0,0.0,8813.0,4897.0,0.0,4897.0,4395.0,0.0,4395.0,0.0,0.0,0.0,7723.0,0.0,7723.0,39.607914,33.583062,0.132706,17.709012,1.920014,0.094704,0.413198,0.292556,0.713998,0.703942,0.703942,0.150845,0.382140,103,0.0,455998,14530,22108,1254,2194,3093,NaN,147,353
2024-10-01,2662638,741049,1921589,178223.0,0.0,178223.0,161789.0,0.0,161789.0,49113.0,0.0,49113.0,71609.0,0.0,71609.0,0.0,0.0,0.0,95421.0,0.0,95421.0,47.364531,32.327339,0.214712,11.110448,0.926863,0.003493,0.569661,0.127130,0.784194,0.779085,0.779085,0.089403,0.224785,1037,0.0,2568250,36314,84026,9133,7987,16517,3830.0,1330,1140
2024-11-01,3145696,911891,2233805,260054.0,0.0,260054.0,52565.0,0.0,52565.0,0.0,0.0,0.0,22656.0,0.0,22656.0,0.0,0.0,0.0,69931.0,0.0,69931.0,41.942546,26.758434,0.162603,10.436864,1.334554,0.008551,13.912247,0.165083,4.502747,4.439960,4.439960,0.634600,1.769257,1097,0.0,3028382,34122,71355,13142,13119,20430,6361.0,4086,3336


In [422]:
res3.n_manual_outbound_calls=res3.n_manual_outbound_calls.fillna(0)
res3.n_inbound_calls=res3.n_inbound_calls.fillna(0)
res3.n_sms_sent=res3.n_sms_sent.fillna(0)

In [423]:
# add week to it and sum weekly expected, caleld, not called
add_weekly1=add5_1.set_index('period').join(lb_hist[['period','call_month']].set_index('period'))
add_weekly1=add_weekly1.reset_index()[['call_month','n_loans_expected_calls_dpd3_15','n_loans_called_dpd3_15']].drop_duplicates()

add_weekly2=add5_2.set_index('period').join(lb_hist[['period','call_month']].set_index('period'))
add_weekly2=add_weekly2.reset_index()[['call_month','n_loans_expected_calls_dpd16_30','n_loans_called_dpd16_30']].drop_duplicates()

add_weekly3=add5_3.set_index('period').join(lb_hist[['period','call_month']].set_index('period'))
add_weekly3=add_weekly3.reset_index()[['call_month','n_loans_expected_calls_dpd0_2','n_loans_called_dpd0_2']].drop_duplicates()


In [424]:
res3['n_loans_expected_calls_dpd0_2']=add_weekly3.groupby('call_month').n_loans_expected_calls_dpd0_2.sum()
res3['n_loans_called_dpd0_2']=add_weekly3.groupby('call_month').n_loans_called_dpd0_2.sum()
res3['perc_loans_called_dpd0_2']=res3.n_loans_called_dpd0_2/res3.n_loans_expected_calls_dpd0_2*100

res3['n_loans_expected_calls_dpd3_15']=add_weekly1.groupby('call_month').n_loans_expected_calls_dpd3_15.sum()
res3['n_loans_called_dpd3_15']=add_weekly1.groupby('call_month').n_loans_called_dpd3_15.sum()
res3['perc_loans_called_dpd3_15']=res3.n_loans_called_dpd3_15/res3.n_loans_expected_calls_dpd3_15*100

res3['n_loans_expected_calls_dpd16_30']=add_weekly2.groupby('call_month').n_loans_expected_calls_dpd16_30.sum()
res3['n_loans_called_dpd16_30']=add_weekly2.groupby('call_month').n_loans_called_dpd16_30.sum()
res3['perc_loans_called_dpd16_30']=res3.n_loans_called_dpd16_30/res3.n_loans_expected_calls_dpd16_30*100



In [425]:
res3.tail(6)

,n_calls,n_queue_calls,n_ivr_calls,n_calls_legal,n_ivr_calls_legal,n_queue_calls_legal,n_calls_mid_priority,n_ivr_calls_mid_priority,n_queue_calls_mid_priority,n_calls_initial,n_ivr_calls_initial,n_queue_calls_initial,n_calls_low_priority,n_ivr_calls_low_priority,n_queue_calls_low_priority,n_calls_high_priority,n_ivr_calls_high_priority,n_queue_calls_high_priority,n_calls_mixed_priority,n_ivr_calls_mixed_priority,n_queue_calls_mixed_priority,perc_calls_busy,perc_calls_no_answer,perc_calls_congestion,perc_calls_answer,perc_calls_abandoned,perc_calls_global,perc_calls_no_such_number,perc_calls_drop,perc_calls_with_disposition1,perc_calls_with_disposition2,perc_calls_with_disposition3,perc_calls_with_disposition1_reschedule,perc_calls_with_disposition2_PTP,n_manual_outbound_calls,n_inbound_calls,n_sms_sent,n_sms_sent_dpd_m3,n_sms_sent_dpd_m1,n_sms_sent_dpd_0,n_sms_sent_dpd_1,n_sms_sent_dpd_3,n_sms_sent_dpd_7,n_sms_sent_dpd_18,n_sms_sent_dpd_25,n_loans_expected_calls_dpd0_2,n_loans_called_dpd0_2,perc_loans_called_dpd0_2,n_loans_expected_calls_dpd3_15,n_loans_called_dpd3_15,perc_loans_called_dpd3_15,n_loans_expected_calls_dpd16_30,n_loans_called_dpd16_30,perc_loans_called_dpd16_30
call_month,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2024-09-01,165780,63682,102098,17051.0,0.0,17051.0,8813.0,0.0,8813.0,4897.0,0.0,4897.0,4395.0,0.0,4395.0,0.0,0.0,0.0,7723.0,0.0,7723.0,39.607914,33.583062,0.132706,17.709012,1.920014,0.094704,0.413198,0.292556,0.713998,0.703942,0.703942,0.150845,0.382140,103,0.0,455998,14530,22108,1254,2194,3093,NaN,147,353,10708,7830,73.122899,4713,1882,39.932103,12828,6261,48.807297
2024-10-01,2662638,741049,1921589,178223.0,0.0,178223.0,161789.0,0.0,161789.0,49113.0,0.0,49113.0,71609.0,0.0,71609.0,0.0,0.0,0.0,95421.0,0.0,95421.0,47.364531,32.327339,0.214712,11.110448,0.926863,0.003493,0.569661,0.127130,0.784194,0.779085,0.779085,0.089403,0.224785,1037,0.0,2568250,36314,84026,9133,7987,16517,3830.0,1330,1140,74943,66705,89.007646,134436,130830,97.317683,163168,153249,93.920989
2024-11-01,3145696,911891,2233805,260054.0,0.0,260054.0,52565.0,0.0,52565.0,0.0,0.0,0.0,22656.0,0.0,22656.0,0.0,0.0,0.0,69931.0,0.0,69931.0,41.942546,26.758434,0.162603,10.436864,1.334554,0.008551,13.912247,0.165083,4.502747,4.439960,4.439960,0.634600,1.769257,1097,0.0,3028382,34122,71355,13142,13119,20430,6361.0,4086,3336,74520,69238,92.911970,147686,137428,93.054183,168154,156836,93.269265


### Upload to SF

In [442]:
res_final=res3

In [444]:
res_final.head(2)

,n_calls,n_queue_calls,n_ivr_calls,n_calls_legal,n_ivr_calls_legal,n_queue_calls_legal,n_calls_mid_priority,n_ivr_calls_mid_priority,n_queue_calls_mid_priority,n_calls_initial,n_ivr_calls_initial,n_queue_calls_initial,n_calls_low_priority,n_ivr_calls_low_priority,n_queue_calls_low_priority,n_calls_high_priority,n_ivr_calls_high_priority,n_queue_calls_high_priority,n_calls_mixed_priority,n_ivr_calls_mixed_priority,n_queue_calls_mixed_priority,perc_calls_busy,perc_calls_no_answer,perc_calls_congestion,perc_calls_answer,perc_calls_abandoned,perc_calls_global,perc_calls_no_such_number,perc_calls_drop,perc_calls_with_disposition1,perc_calls_with_disposition2,perc_calls_with_disposition3,perc_calls_with_disposition1_reschedule,perc_calls_with_disposition2_PTP,n_manual_outbound_calls,n_inbound_calls,n_sms_sent,n_sms_sent_dpd_m3,n_sms_sent_dpd_m1,n_sms_sent_dpd_0,n_sms_sent_dpd_1,n_sms_sent_dpd_3,n_sms_sent_dpd_7,n_sms_sent_dpd_18,n_sms_sent_dpd_25,n_loans_expected_calls_dpd0_2,n_loans_called_dpd0_2,perc_loans_called_dpd0_2,n_loans_expected_calls_dpd3_15,n_loans_called_dpd3_15,perc_loans_called_dpd3_15,n_loans_expected_calls_dpd16_30,n_loans_called_dpd16_30,perc_loans_called_dpd16_30
call_month,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2024-09-01,165780,63682,102098,17051.0,0.0,17051.0,8813.0,0.0,8813.0,4897.0,0.0,4897.0,4395.0,0.0,4395.0,0.0,0.0,0.0,7723.0,0.0,7723.0,39.607914,33.583062,0.132706,17.709012,1.920014,0.094704,0.413198,0.292556,0.713998,0.703942,0.703942,0.150845,0.382140,103,0.0,455998,14530,22108,1254,2194,3093,NaN,147,353,10708,7830,73.122899,4713,1882,39.932103,12828,6261,48.807297
2024-10-01,2662638,741049,1921589,178223.0,0.0,178223.0,161789.0,0.0,161789.0,49113.0,0.0,49113.0,71609.0,0.0,71609.0,0.0,0.0,0.0,95421.0,0.0,95421.0,47.364531,32.327339,0.214712,11.110448,0.926863,0.003493,0.569661,0.127130,0.784194,0.779085,0.779085,0.089403,0.224785,1037,0.0,2568250,36314,84026,9133,7987,16517,3830.0,1330,1140,74943,66705,89.007646,134436,130830,97.317683,163168,153249,93.920989


In [450]:
res_final=res_final.iloc[1:,:]
res_final

,n_calls,n_queue_calls,n_ivr_calls,n_calls_legal,n_ivr_calls_legal,n_queue_calls_legal,n_calls_mid_priority,n_ivr_calls_mid_priority,n_queue_calls_mid_priority,n_calls_initial,n_ivr_calls_initial,n_queue_calls_initial,n_calls_low_priority,n_ivr_calls_low_priority,n_queue_calls_low_priority,n_calls_high_priority,n_ivr_calls_high_priority,n_queue_calls_high_priority,n_calls_mixed_priority,n_ivr_calls_mixed_priority,n_queue_calls_mixed_priority,perc_calls_busy,perc_calls_no_answer,perc_calls_congestion,perc_calls_answer,perc_calls_abandoned,perc_calls_global,perc_calls_no_such_number,perc_calls_drop,perc_calls_with_disposition1,perc_calls_with_disposition2,perc_calls_with_disposition3,perc_calls_with_disposition1_reschedule,perc_calls_with_disposition2_PTP,n_manual_outbound_calls,n_inbound_calls,n_sms_sent,n_sms_sent_dpd_m3,n_sms_sent_dpd_m1,n_sms_sent_dpd_0,n_sms_sent_dpd_1,n_sms_sent_dpd_3,n_sms_sent_dpd_7,n_sms_sent_dpd_18,n_sms_sent_dpd_25,n_loans_expected_calls_dpd0_2,n_loans_called_dpd0_2,perc_loans_called_dpd0_2,n_loans_expected_calls_dpd3_15,n_loans_called_dpd3_15,perc_loans_called_dpd3_15,n_loans_expected_calls_dpd16_30,n_loans_called_dpd16_30,perc_loans_called_dpd16_30
call_month,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2024-10-01,2662638,741049,1921589,178223.0,0.0,178223.0,161789.0,0.0,161789.0,49113.0,0.0,49113.0,71609.0,0.0,71609.0,0.0,0.0,0.0,95421.0,0.0,95421.0,47.364531,32.327339,0.214712,11.110448,0.926863,0.003493,0.569661,0.127130,0.784194,0.779085,0.779085,0.089403,0.224785,1037,0.0,2568250,36314,84026,9133,7987,16517,3830.0,1330,1140,74943,66705,89.007646,134436,130830,97.317683,163168,153249,93.920989
2024-11-01,3145696,911891,2233805,260054.0,0.0,260054.0,52565.0,0.0,52565.0,0.0,0.0,0.0,22656.0,0.0,22656.0,0.0,0.0,0.0,69931.0,0.0,69931.0,41.942546,26.758434,0.162603,10.436864,1.334554,0.008551,13.912247,0.165083,4.502747,4.439960,4.439960,0.634600,1.769257,1097,0.0,3028382,34122,71355,13142,13119,20430,6361.0,4086,3336,74520,69238,92.911970,147686,137428,93.054183,168154,156836,93.269265


In [452]:
res_final=res_final.reset_index()

In [454]:
res_final=res_final.rename(columns={'call_month':'date_month'})

In [456]:
res_final.head(2)

,date_month,n_calls,n_queue_calls,n_ivr_calls,n_calls_legal,n_ivr_calls_legal,n_queue_calls_legal,n_calls_mid_priority,n_ivr_calls_mid_priority,n_queue_calls_mid_priority,n_calls_initial,n_ivr_calls_initial,n_queue_calls_initial,n_calls_low_priority,n_ivr_calls_low_priority,n_queue_calls_low_priority,n_calls_high_priority,n_ivr_calls_high_priority,n_queue_calls_high_priority,n_calls_mixed_priority,n_ivr_calls_mixed_priority,n_queue_calls_mixed_priority,perc_calls_busy,perc_calls_no_answer,perc_calls_congestion,perc_calls_answer,perc_calls_abandoned,perc_calls_global,perc_calls_no_such_number,perc_calls_drop,perc_calls_with_disposition1,perc_calls_with_disposition2,perc_calls_with_disposition3,perc_calls_with_disposition1_reschedule,perc_calls_with_disposition2_PTP,n_manual_outbound_calls,n_inbound_calls,n_sms_sent,n_sms_sent_dpd_m3,n_sms_sent_dpd_m1,n_sms_sent_dpd_0,n_sms_sent_dpd_1,n_sms_sent_dpd_3,n_sms_sent_dpd_7,n_sms_sent_dpd_18,n_sms_sent_dpd_25,n_loans_expected_calls_dpd0_2,n_loans_called_dpd0_2,perc_loans_called_dpd0_2,n_loans_expected_calls_dpd3_15,n_loans_called_dpd3_15,perc_loans_called_dpd3_15,n_loans_expected_calls_dpd16_30,n_loans_called_dpd16_30,perc_loans_called_dpd16_30
0,2024-10-01,2662638,741049,1921589,178223.0,0.0,178223.0,161789.0,0.0,161789.0,49113.0,0.0,49113.0,71609.0,0.0,71609.0,0.0,0.0,0.0,95421.0,0.0,95421.0,47.364531,32.327339,0.214712,11.110448,0.926863,0.003493,0.569661,0.127130,0.784194,0.779085,0.779085,0.089403,0.224785,1037,0.0,2568250,36314,84026,9133,7987,16517,3830.0,1330,1140,74943,66705,89.007646,134436,130830,97.317683,163168,153249,93.920989
1,2024-11-01,3145696,911891,2233805,260054.0,0.0,260054.0,52565.0,0.0,52565.0,0.0,0.0,0.0,22656.0,0.0,22656.0,0.0,0.0,0.0,69931.0,0.0,69931.0,41.942546,26.758434,0.162603,10.436864,1.334554,0.008551,13.912247,0.165083,4.502747,4.439960,4.439960,0.634600,1.769257,1097,0.0,3028382,34122,71355,13142,13119,20430,6361.0,4086,3336,74520,69238,92.911970,147686,137428,93.054183,168154,156836,93.269265


In [458]:
my_user="AMIT_S"
my_pass="Amit2525!!"
my_account="xukvgcv-xm56910"
my_warehouse='ANALYTICS_WH'
my_db='ghana_prod'
table_name='collections_blood_tests_monthly'

In [460]:
import snowflake.connector
from snowflake.sqlalchemy import URL
from sqlalchemy import create_engine

In [462]:
engine = create_engine(URL(user=my_user,password=my_pass,account=my_account,warehouse=my_warehouse,database=my_db))
connection = engine.connect()

In [464]:
res_final=res_final.round(2)

In [466]:
# # # FIRST UPLOAD ONLY
res_final.to_sql(table_name, connection, schema='risk_reports',if_exists='append', index=False)


2